In [1]:
import os
import pickle
import string

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW11OT-MakeConcordance-1Kings.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""


import os
import pickle
import string

from BibleWordList2 import *

BUILDAWIP = 1
BUILDNWIV = 1
BUILDPNIP = 1
BUILDNWIC = 1
BUILDNLIW = 1
BUILDBPIP = 1

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

FirstKings01 = [
    ['1Kings','1Ki','01','001','1',"Now king David was old \emph{and} stricken in years; and they covered him with clothes, but he gat no heat."],
['1Kings','1Ki','01','002','2',"Wherefore his servants said unto him, Let there be sought for my lord the king a young virgin: and let her stand before the king, and let her cherish him, and let her lie in thy bosom, that my lord the king may get heat."],
['1Kings','1Ki','01','003','3',"So they sought for a fair damsel throughout all the coasts of Israel, and found Abishag a Shunammite, and brought her to the king."],
['1Kings','1Ki','01','004','4',"And the damsel \emph{was} very fair, and cherished the king, and ministered to him: but the king knew her not."],
['1Kings','1Ki','01','005','5',"Then Adonijah the son of Haggith exalted himself, saying, I will be king: and he prepared him chariots and horsemen, and fifty men to run before him."],
['1Kings','1Ki','01','006','6',"And his father had not displeased him at any time in saying, Why hast thou done so? and he also \emph{was} \emph{a} very goodly \emph{man}; and \emph{his} \emph{mother} bare him after Absalom."],
['1Kings','1Ki','01','007','7',"And he conferred with Joab the son of Zeruiah, and with Abiathar the priest: and they following Adonijah helped \emph{him}."],
['1Kings','1Ki','01','008','8',"But Zadok the priest, and Benaiah the son of Jehoiada, and Nathan the prophet, and Shimei, and Rei, and the mighty men which \emph{belonged} to David, were not with Adonijah."],
['1Kings','1Ki','01','009','9',"And Adonijah slew sheep and oxen and fat cattle by the stone of Zoheleth, which \emph{is} by En-rogel, and called all his brethren the king's sons, and all the men of Judah the king's servants:"],
['1Kings','1Ki','01','010','10',"But Nathan the prophet, and Benaiah, and the mighty men, and Solomon his brother, he called not."],
['1Kings','1Ki','01','011','11',"Wherefore Nathan spake unto Bath-sheba the mother of Solomon, saying, Hast thou not heard that Adonijah the son of Haggith doth reign, and David our lord knoweth \emph{it} not?"],
['1Kings','1Ki','01','012','12',"Now therefore come, let me, I pray thee, give thee counsel, that thou mayest save thine own life, and the life of thy son Solomon."],
['1Kings','1Ki','01','013','13',"Go and get thee in unto king David, and say unto him, Didst not thou, my lord, O king, swear unto thine handmaid, saying, Assuredly Solomon thy son shall reign after me, and he shall sit upon my throne? why then doth Adonijah reign?"],
['1Kings','1Ki','01','014','14',"Behold, while thou yet talkest there with the king, I also will come in after thee, and confirm thy words."],
['1Kings','1Ki','01','015','15',"And Bath-sheba went in unto the king into the chamber: and the king was very old; and Abishag the Shunammite ministered unto the king."],
['1Kings','1Ki','01','016','16',"And Bath-sheba bowed, and did obeisance unto the king. And the king said, What wouldest thou?"],
['1Kings','1Ki','01','017','17',"And she said unto him, My lord, thou swarest by the LORD thy God unto thine handmaid, \emph{saying}, Assuredly Solomon thy son shall reign after me, and he shall sit upon my throne."],
['1Kings','1Ki','01','018','18',"And now, behold, Adonijah reigneth; and now, my lord the king, thou knowest \emph{it} not:"],
['1Kings','1Ki','01','019','19',"And he hath slain oxen and fat cattle and sheep in abundance, and hath called all the sons of the king, and Abiathar the priest, and Joab the captain of the host: but Solomon thy servant hath he not called."],
['1Kings','1Ki','01','020','20',"And thou, my lord, O king, the eyes of all Israel \emph{are} upon thee, that thou shouldest tell them who shall sit on the throne of my lord the king after him."],
['1Kings','1Ki','01','021','21',"Otherwise it shall come to pass, when my lord the king shall sleep with his fathers, that I and my son Solomon shall be counted offenders."],
['1Kings','1Ki','01','022','22',"And, lo, while she yet talked with the king, Nathan the prophet also came in."],
['1Kings','1Ki','01','023','23',"And they told the king, saying, Behold Nathan the prophet. And when he was come in before the king, he bowed himself before the king with his face to the ground."],
['1Kings','1Ki','01','024','24',"And Nathan said, My lord, O king, hast thou said, Adonijah shall reign after me, and he shall sit upon my throne?"],
['1Kings','1Ki','01','025','25',"For he is gone down this day, and hath slain oxen and fat cattle and sheep in abundance, and hath called all the king's sons, and the captains of the host, and Abiathar the priest; and, behold, they eat and drink before him, and say, God save king Adonijah."],
['1Kings','1Ki','01','026','26',"But me, \emph{even} me thy servant, and Zadok the priest, and Benaiah the son of Jehoiada, and thy servant Solomon, hath he not called."],
['1Kings','1Ki','01','027','27',"Is this thing done by my lord the king, and thou hast not shewed \emph{it} unto thy servant, who should sit on the throne of my lord the king after him?"],
['1Kings','1Ki','01','028','28',"Then king David answered and said, Call me Bath-sheba. And she came into the king's presence, and stood before the king."],
['1Kings','1Ki','01','029','29',"And the king sware, and said, \emph{As} the LORD liveth, that hath redeemed my soul out of all distress,"],
['1Kings','1Ki','01','030','30',"Even as I sware unto thee by the LORD God of Israel, saying, Assuredly Solomon thy son shall reign after me, and he shall sit upon my throne in my stead; even so will I certainly do this day."],
['1Kings','1Ki','01','031','31',"Then Bath-sheba bowed with \emph{her} face to the earth, and did reverence to the king, and said, Let my lord king David live for ever."],
['1Kings','1Ki','01','032','32',"And king David said, Call me Zadok the priest, and Nathan the prophet, and Benaiah the son of Jehoiada. And they came before the king."],
['1Kings','1Ki','01','033','33',"The king also said unto them, Take with you the servants of your lord, and cause Solomon my son to ride upon mine own mule, and bring him down to Gihon:"],
['1Kings','1Ki','01','034','34',"And let Zadok the priest and Nathan the prophet anoint him there king over Israel: and blow ye with the trumpet, and say, God save king Solomon."],
['1Kings','1Ki','01','035','35',"Then ye shall come up after him, that he may come and sit upon my throne; for he shall be king in my stead: and I have appointed him to be ruler over Israel and over Judah."],
['1Kings','1Ki','01','036','36',"And Benaiah the son of Jehoiada answered the king, and said, Amen: the LORD God of my lord the king say so \emph{too}."],
['1Kings','1Ki','01','037','37',"As the LORD hath been with my lord the king, even so be he with Solomon, and make his throne greater than the throne of my lord king David."],
['1Kings','1Ki','01','038','38',"So Zadok the priest, and Nathan the prophet, and Benaiah the son of Jehoiada, and the Cherethites, and the Pelethites, went down, and caused Solomon to ride upon king David's mule, and brought him to Gihon."],
['1Kings','1Ki','01','039','39',"And Zadok the priest took an horn of oil out of the tabernacle, and anointed Solomon. And they blew the trumpet; and all the people said, God save king Solomon."],
['1Kings','1Ki','01','040','40',"And all the people came up after him, and the people piped with pipes, and rejoiced with great joy, so that the earth rent with the sound of them."],
['1Kings','1Ki','01','041','41',"And Adonijah and all the guests that \emph{were} with him heard \emph{it} as they had made an end of eating. And when Joab heard the sound of the trumpet, he said, Wherefore \emph{is} \emph{this} noise of the city being in an uproar?"],
['1Kings','1Ki','01','042','42',"And while he yet spake, behold, Jonathan the son of Abiathar the priest came: and Adonijah said unto him, Come in; for thou \emph{art} a valiant man, and bringest good tidings."],
['1Kings','1Ki','01','043','43',"And Jonathan answered and said to Adonijah, Verily our lord king David hath made Solomon king."],
['1Kings','1Ki','01','044','44',"And the king hath sent with him Zadok the priest, and Nathan the prophet, and Benaiah the son of Jehoiada, and the Cherethites, and the Pelethites, and they have caused him to ride upon the king's mule:"],
['1Kings','1Ki','01','045','45',"And Zadok the priest and Nathan the prophet have anointed him king in Gihon: and they are come up from thence rejoicing, so that the city rang again. This \emph{is} the noise that ye have heard."],
['1Kings','1Ki','01','046','46',"And also Solomon sitteth on the throne of the kingdom."],
['1Kings','1Ki','01','047','47',"And moreover the king's servants came to bless our lord king David, saying, God make the name of Solomon better than thy name, and make his throne greater than thy throne. And the king bowed himself upon the bed."],
['1Kings','1Ki','01','048','48',"And also thus said the king, Blessed \emph{be} the LORD God of Israel, which hath given \emph{one} to sit on my throne this day, mine eyes even seeing \emph{it}."],
['1Kings','1Ki','01','049','49',"And all the guests that \emph{were} with Adonijah were afraid, and rose up, and went every man his way."],
['1Kings','1Ki','01','050','50',"And Adonijah feared because of Solomon, and arose, and went, and caught hold on the horns of the altar."],
['1Kings','1Ki','01','051','51',"And it was told Solomon, saying, Behold, Adonijah feareth king Solomon: for, lo, he hath caught hold on the horns of the altar, saying, Let king Solomon swear unto me to day that he will not slay his servant with the sword."],
['1Kings','1Ki','01','052','52',"And Solomon said, If he will shew himself a worthy man, there shall not an hair of him fall to the earth: but if wickedness shall be found in him, he shall die."],
['1Kings','1Ki','01','053','53',"So king Solomon sent, and they brought him down from the altar. And he came and bowed himself to king Solomon: and Solomon said unto him, Go to thine house."]
]

FirstKings02 = [
    ['1Kings','1Ki','02','001','1',"Now the days of David drew nigh that he should die; and he charged Solomon his son, saying,"],
['1Kings','1Ki','02','002','2',"I go the way of all the earth: be thou strong therefore, and shew thyself a man;"],
['1Kings','1Ki','02','003','3',"And keep the charge of the LORD thy God, to walk in his ways, to keep his statutes, and his commandments, and his judgments, and his testimonies, as it is written in the law of Moses, that thou mayest prosper in all that thou doest, and whithersoever thou turnest thyself:"],
['1Kings','1Ki','02','004','4',"That the LORD may continue his word which he spake concerning me, saying, If thy children take heed to their way, to walk before me in truth with all their heart and with all their soul, there shall not fail thee (said he) a man on the throne of Israel."],
['1Kings','1Ki','02','005','5',"Moreover thou knowest also what Joab the son of Zeruiah did to me, \emph{and} what he did to the two captains of the hosts of Israel, unto Abner the son of Ner, and unto Amasa the son of Jether, whom he slew, and shed the blood of war in peace, and put the blood of war upon his girdle that \emph{was} about his loins, and in his shoes that \emph{were} on his feet."],
['1Kings','1Ki','02','006','6',"Do therefore according to thy wisdom, and let not his hoar head go down to the grave in peace."],
['1Kings','1Ki','02','007','7',"But shew kindness unto the sons of Barzillai the Gileadite, and let them be of those that eat at thy table: for so they came to me when I fled because of Absalom thy brother."],
['1Kings','1Ki','02','008','8',"And, behold, \emph{thou} \emph{hast} with thee Shimei the son of Gera, a Benjamite of Bahurim, which cursed me with a grievous curse in the day when I went to Mahanaim: but he came down to meet me at Jordan, and I sware to him by the LORD, saying, I will not put thee to death with the sword."],
['1Kings','1Ki','02','009','9',"Now therefore hold him not guiltless: for thou \emph{art} a wise man, and knowest what thou oughtest to do unto him; but his hoar head bring thou down to the grave with blood."],
['1Kings','1Ki','02','010','10',"So David slept with his fathers, and was buried in the city of David."],
['1Kings','1Ki','02','011','11',"And the days that David reigned over Israel \emph{were} forty years: seven years reigned he in Hebron, and thirty and three years reigned he in Jerusalem."],
['1Kings','1Ki','02','012','12',"Then sat Solomon upon the throne of David his father; and his kingdom was established greatly."],
['1Kings','1Ki','02','013','13',"And Adonijah the son of Haggith came to Bath-sheba the mother of Solomon. And she said, Comest thou peaceably? And he said, Peaceably."],
['1Kings','1Ki','02','014','14',"He said moreover, I have somewhat to say unto thee. And she said, Say on."],
['1Kings','1Ki','02','015','15',"And he said, Thou knowest that the kingdom was mine, and \emph{that} all Israel set their faces on me, that I should reign: howbeit the kingdom is turned about, and is become my brother's: for it was his from the LORD."],
['1Kings','1Ki','02','016','16',"And now I ask one petition of thee, deny me not. And she said unto him, Say on."],
['1Kings','1Ki','02','017','17',"And he said, Speak, I pray thee, unto Solomon the king, (for he will not say thee nay,) that he give me Abishag the Shunammite to wife."],
['1Kings','1Ki','02','018','18',"And Bath-sheba said, Well; I will speak for thee unto the king."],
['1Kings','1Ki','02','019','19',"Bath-sheba therefore went unto king Solomon, to speak unto him for Adonijah. And the king rose up to meet her, and bowed himself unto her, and sat down on his throne, and caused a seat to be set for the king's mother; and she sat on his right hand."],
['1Kings','1Ki','02','020','20',"Then she said, I desire one small petition of thee; \emph{I} \emph{pray} \emph{thee}, say me not nay. And the king said unto her, Ask on, my mother: for I will not say thee nay."],
['1Kings','1Ki','02','021','21',"And she said, Let Abishag the Shunammite be given to Adonijah thy brother to wife."],
['1Kings','1Ki','02','022','22',"And king Solomon answered and said unto his mother, And why dost thou ask Abishag the Shunammite for Adonijah? ask for him the kingdom also; for he \emph{is} mine elder brother; even for him, and for Abiathar the priest, and for Joab the son of Zeruiah."],
['1Kings','1Ki','02','023','23',"Then king Solomon sware by the LORD, saying, God do so to me, and more also, if Adonijah have not spoken this word against his own life."],
['1Kings','1Ki','02','024','24',"Now therefore, \emph{as} the LORD liveth, which hath established me, and set me on the throne of David my father, and who hath made me an house, as he promised, Adonijah shall be put to death this day."],
['1Kings','1Ki','02','025','25',"And king Solomon sent by the hand of Benaiah the son of Jehoiada; and he fell upon him that he died."],
['1Kings','1Ki','02','026','26',"And unto Abiathar the priest said the king, Get thee to Anathoth, unto thine own fields; for thou \emph{art} worthy of death: but I will not at this time put thee to death, because thou barest the ark of the Lord GOD before David my father, and because thou hast been afflicted in all wherein my father was afflicted."],
['1Kings','1Ki','02','027','27',"So Solomon thrust out Abiathar from being priest unto the LORD; that he might fulfil the word of the LORD, which he spake concerning the house of Eli in Shiloh."],
['1Kings','1Ki','02','028','28',"Then tidings came to Joab: for Joab had turned after Adonijah, though he turned not after Absalom. And Joab fled unto the tabernacle of the LORD, and caught hold on the horns of the altar."],
['1Kings','1Ki','02','029','29',"And it was told king Solomon that Joab was fled unto the tabernacle of the LORD; and, behold, \emph{he} \emph{is} by the altar. Then Solomon sent Benaiah the son of Jehoiada, saying, Go, fall upon him."],
['1Kings','1Ki','02','030','30',"And Benaiah came to the tabernacle of the LORD, and said unto him, Thus saith the king, Come forth. And he said, Nay; but I will die here. And Benaiah brought the king word again, saying, Thus said Joab, and thus he answered me."],
['1Kings','1Ki','02','031','31',"And the king said unto him, Do as he hath said, and fall upon him, and bury him; that thou mayest take away the innocent blood, which Joab shed, from me, and from the house of my father."],
['1Kings','1Ki','02','032','32',"And the LORD shall return his blood upon his own head, who fell upon two men more righteous and better than he, and slew them with the sword, my father David not knowing \emph{thereof,} \emph{to} \emph{wit}, Abner the son of Ner, captain of the host of Israel, and Amasa the son of Jether, captain of the host of Judah."],
['1Kings','1Ki','02','033','33',"Their blood shall therefore return upon the head of Joab, and upon the head of his seed for ever: but upon David, and upon his seed, and upon his house, and upon his throne, shall there be peace for ever from the LORD."],
['1Kings','1Ki','02','034','34',"So Benaiah the son of Jehoiada went up, and fell upon him, and slew him: and he was buried in his own house in the wilderness."],
['1Kings','1Ki','02','035','35',"And the king put Benaiah the son of Jehoiada in his room over the host: and Zadok the priest did the king put in the room of Abiathar."],
['1Kings','1Ki','02','036','36',"And the king sent and called for Shimei, and said unto him, Build thee an house in Jerusalem, and dwell there, and go not forth thence any whither."],
['1Kings','1Ki','02','037','37',"For it shall be, \emph{that} on the day thou goest out, and passest over the brook Kidron, thou shalt know for certain that thou shalt surely die: thy blood shall be upon thine own head."],
['1Kings','1Ki','02','038','38',"And Shimei said unto the king, The saying \emph{is} good: as my lord the king hath said, so will thy servant do. And Shimei dwelt in Jerusalem many days."],
['1Kings','1Ki','02','039','39',"And it came to pass at the end of three years, that two of the servants of Shimei ran away unto Achish son of Maachah king of Gath. And they told Shimei, saying, Behold, thy servants \emph{be} in Gath."],
['1Kings','1Ki','02','040','40',"And Shimei arose, and saddled his ass, and went to Gath to Achish to seek his servants: and Shimei went, and brought his servants from Gath."],
['1Kings','1Ki','02','041','41',"And it was told Solomon that Shimei had gone from Jerusalem to Gath, and was come again."],
['1Kings','1Ki','02','042','42',"And the king sent and called for Shimei, and said unto him, Did I not make thee to swear by the LORD, and protested unto thee, saying, Know for a certain, on the day thou goest out, and walkest abroad any whither, that thou shalt surely die? and thou saidst unto me, The word \emph{that} I have heard \emph{is} good."],
['1Kings','1Ki','02','043','43',"Why then hast thou not kept the oath of the LORD, and the commandment that I have charged thee with?"],
['1Kings','1Ki','02','044','44',"The king said moreover to Shimei, Thou knowest all the wickedness which thine heart is privy to, that thou didst to David my father: therefore the LORD shall return thy wickedness upon thine own head;"],
['1Kings','1Ki','02','045','45',"And king Solomon \emph{shall} \emph{be} blessed, and the throne of David shall be established before the LORD for ever."],
['1Kings','1Ki','02','046','46',"So the king commanded Benaiah the son of Jehoiada; which went out, and fell upon him, that he died. And the kingdom was established in the hand of Solomon."]
]

FirstKings03 = [
    ['1Kings','1Ki','03','001','1',"And Solomon made affinity with Pharaoh king of Egypt, and took Pharaoh's daughter, and brought her into the city of David, until he had made an end of building his own house, and the house of the LORD, and the wall of Jerusalem round about."],
['1Kings','1Ki','03','002','2',"Only the people sacrificed in high places, because there was no house built unto the name of the LORD, until those days."],
['1Kings','1Ki','03','003','3',"And Solomon loved the LORD, walking in the statutes of David his father: only he sacrificed and burnt incense in high places."],
['1Kings','1Ki','03','004','4',"And the king went to Gibeon to sacrifice there; for that \emph{was} the great high place: a thousand burnt offerings did Solomon offer upon that altar."],
['1Kings','1Ki','03','005','5',"In Gibeon the LORD appeared to Solomon in a dream by night: and God said, Ask what I shall give thee."],
['1Kings','1Ki','03','006','6',"And Solomon said, Thou hast shewed unto thy servant David my father great mercy, according as he walked before thee in truth, and in righteousness, and in uprightness of heart with thee; and thou hast kept for him this great kindness, that thou hast given him a son to sit on his throne, as \emph{it} \emph{is} this day."],
['1Kings','1Ki','03','007','7',"And now, O LORD my God, thou hast made thy servant king instead of David my father: and I \emph{am} \emph{but} a little child: I know not \emph{how} to go out or come in."],
['1Kings','1Ki','03','008','8',"And thy servant \emph{is} in the midst of thy people which thou hast chosen, a great people, that cannot be numbered nor counted for multitude."],
['1Kings','1Ki','03','009','9',"Give therefore thy servant an understanding heart to judge thy people, that I may discern between good and bad: for who is able to judge this thy so great a people?"],
['1Kings','1Ki','03','010','10',"And the speech pleased the Lord, that Solomon had asked this thing."],
['1Kings','1Ki','03','011','11',"And God said unto him, Because thou hast asked this thing, and hast not asked for thyself long life; neither hast asked riches for thyself, nor hast asked the life of thine enemies; but hast asked for thyself understanding to discern judgment;"],
['1Kings','1Ki','03','012','12',"Behold, I have done according to thy words: lo, I have given thee a wise and an understanding heart; so that there was none like thee before thee, neither after thee shall any arise like unto thee."],
['1Kings','1Ki','03','013','13',"And I have also given thee that which thou hast not asked, both riches, and honour: so that there shall not be any among the kings like unto thee all thy days."],
['1Kings','1Ki','03','014','14',"And if thou wilt walk in my ways, to keep my statutes and my commandments, as thy father David did walk, then I will lengthen thy days."],
['1Kings','1Ki','03','015','15',"And Solomon awoke; and, behold, \emph{it} \emph{was} a dream. And he came to Jerusalem, and stood before the ark of the covenant of the LORD, and offered up burnt offerings, and offered peace offerings, and made a feast to all his servants."],
['1Kings','1Ki','03','016','16',"Then came there two women, \emph{that} \emph{were} harlots, unto the king, and stood before him."],
['1Kings','1Ki','03','017','17',"And the one woman said, O my lord, I and this woman dwell in one house; and I was delivered of a child with her in the house."],
['1Kings','1Ki','03','018','18',"And it came to pass the third day after that I was delivered, that this woman was delivered also: and we \emph{were} together; \emph{there} \emph{was} no stranger with us in the house, save we two in the house."],
['1Kings','1Ki','03','019','19',"And this woman's child died in the night; because she overlaid it."],
['1Kings','1Ki','03','020','20',"And she arose at midnight, and took my son from beside me, while thine handmaid slept, and laid it in her bosom, and laid her dead child in my bosom."],
['1Kings','1Ki','03','021','21',"And when I rose in the morning to give my child suck, behold, it was dead: but when I had considered it in the morning, behold, it was not my son, which I did bear."],
['1Kings','1Ki','03','022','22',"And the other woman said, Nay; but the living \emph{is} my son, and the dead \emph{is} thy son. And this said, No; but the dead \emph{is} thy son, and the living \emph{is} my son. Thus they spake before the king."],
['1Kings','1Ki','03','023','23',"Then said the king, The one saith, This \emph{is} my son that liveth, and thy son \emph{is} the dead: and the other saith, Nay; but thy son \emph{is} the dead, and my son \emph{is} the living."],
['1Kings','1Ki','03','024','24',"And the king said, Bring me a sword. And they brought a sword before the king."],
['1Kings','1Ki','03','025','25',"And the king said, Divide the living child in two, and give half to the one, and half to the other."],
['1Kings','1Ki','03','026','26',"Then spake the woman whose the living child \emph{was} unto the king, for her bowels yearned upon her son, and she said, O my lord, give her the living child, and in no wise slay it. But the other said, Let it be neither mine nor thine, \emph{but} divide \emph{it}."],
['1Kings','1Ki','03','027','27',"Then the king answered and said, Give her the living child, and in no wise slay it: she \emph{is} the mother thereof."],
['1Kings','1Ki','03','028','28',"And all Israel heard of the judgment which the king had judged; and they feared the king: for they saw that the wisdom of God \emph{was} in him, to do judgment."]
]

FirstKings04 = [
    ['1Kings','1Ki','04','001','1',"So king Solomon was king over all Israel."],
['1Kings','1Ki','04','002','2',"And these \emph{were} the princes which he had; Azariah the son of Zadok the priest,"],
['1Kings','1Ki','04','003','3',"Elihoreph and Ahiah, the sons of Shisha, scribes; Jehoshaphat the son of Ahilud, the recorder."],
['1Kings','1Ki','04','004','4',"And Benaiah the son of Jehoiada \emph{was} over the host: and Zadok and Abiathar \emph{were} the priests:"],
['1Kings','1Ki','04','005','5',"And Azariah the son of Nathan \emph{was} over the officers: and Zabud the son of Nathan \emph{was} principal officer, \emph{and} the king's friend:"],
['1Kings','1Ki','04','006','6',"And Ahishar \emph{was} over the household: and Adoniram the son of Abda \emph{was} over the tribute."],
['1Kings','1Ki','04','007','7',"And Solomon had twelve officers over all Israel, which provided victuals for the king and his household: each man his month in a year made provision."],
['1Kings','1Ki','04','008','8',"And these \emph{are} their names: The son of Hur, in mount Ephraim:"],
['1Kings','1Ki','04','009','9',"The son of Dekar, in Makaz, and in Shaalbim, and Beth-shemesh, and Elon-beth-hanan:"],
['1Kings','1Ki','04','010','10',"The son of Hesed, in Aruboth; to him \emph{pertained} Sochoh, and all the land of Hepher:"],
['1Kings','1Ki','04','011','11',"The son of Abinadab, in all the region of Dor; which had Taphath the daughter of Solomon to wife:"],
['1Kings','1Ki','04','012','12',"Baana the son of Ahilud; \emph{to} \emph{him} \emph{pertained} Taanach and Megiddo, and all Beth-shean, which \emph{is} by Zartanah beneath Jezreel, from Beth-shean to Abel-meholah, \emph{even} unto \emph{the} \emph{place} \emph{that} \emph{is} beyond Jokneam:"],
['1Kings','1Ki','04','013','13',"The son of Geber, in Ramoth-gilead; to him \emph{pertained} the towns of Jair the son of Manasseh, which \emph{are} in Gilead; to him \emph{also} \emph{pertained} the region of Argob, which \emph{is} in Bashan, threescore great cities with walls and brasen bars:"],
['1Kings','1Ki','04','014','14',"Ahinadab the son of Iddo \emph{had} Mahanaim:"],
['1Kings','1Ki','04','015','15',"Ahimaaz \emph{was} in Naphtali; he also took Basmath the daughter of Solomon to wife:"],
['1Kings','1Ki','04','016','16',"Baanah the son of Hushai \emph{was} in Asher and in Aloth:"],
['1Kings','1Ki','04','017','17',"Jehoshaphat the son of Paruah, in Issachar:"],
['1Kings','1Ki','04','018','18',"Shimei the son of Elah, in Benjamin:"],
['1Kings','1Ki','04','019','19',"Geber the son of Uri \emph{was} in the country of Gilead, \emph{in} the country of Sihon king of the Amorites, and of Og king of Bashan; and \emph{he} \emph{was} the only officer which \emph{was} in the land."],
['1Kings','1Ki','04','020','20',"Judah and Israel \emph{were} many, as the sand which \emph{is} by the sea in multitude, eating and drinking, and making merry."],
['1Kings','1Ki','04','021','21',"And Solomon reigned over all kingdoms from the river unto the land of the Philistines, and unto the border of Egypt: they brought presents, and served Solomon all the days of his life."],
['1Kings','1Ki','04','022','22',"And Solomon's provision for one day was thirty measures of fine flour, and threescore measures of meal,"],
['1Kings','1Ki','04','023','23',"Ten fat oxen, and twenty oxen out of the pastures, and an hundred sheep, beside harts, and roebucks, and fallowdeer, and fatted fowl."],
['1Kings','1Ki','04','024','24',"For he had dominion over all \emph{the} \emph{region} on this side the river, from Tiphsah even to Azzah, over all the kings on this side the river: and he had peace on all sides round about him."],
['1Kings','1Ki','04','025','25',"And Judah and Israel dwelt safely, every man under his vine and under his fig tree, from Dan even to Beer-sheba, all the days of Solomon."],
['1Kings','1Ki','04','026','26',"And Solomon had forty thousand stalls of horses for his chariots, and twelve thousand horsemen."],
['1Kings','1Ki','04','027','27',"And those officers provided victual for king Solomon, and for all that came unto king Solomon's table, every man in his month: they lacked nothing."],
['1Kings','1Ki','04','028','28',"Barley also and straw for the horses and dromedaries brought they unto the place where \emph{the} \emph{officers} were, every man according to his charge."],
['1Kings','1Ki','04','029','29',"And God gave Solomon wisdom and understanding exceeding much, and largeness of heart, even as the sand that \emph{is} on the sea shore."],
['1Kings','1Ki','04','030','30',"And Solomon's wisdom excelled the wisdom of all the children of the east country, and all the wisdom of Egypt."],
['1Kings','1Ki','04','031','31',"For he was wiser than all men; than Ethan the Ezrahite, and Heman, and Chalcol, and Darda, the sons of Mahol: and his fame was in all nations round about."],
['1Kings','1Ki','04','032','32',"And he spake three thousand proverbs: and his songs were a thousand and five."],
['1Kings','1Ki','04','033','33',"And he spake of trees, from the cedar tree that \emph{is} in Lebanon even unto the hyssop that springeth out of the wall: he spake also of beasts, and of fowl, and of creeping things, and of fishes."],
['1Kings','1Ki','04','034','34',"And there came of all people to hear the wisdom of Solomon, from all kings of the earth, which had heard of his wisdom."]
        ]

FirstKings05 = [
    ['1Kings','1Ki','05','001','1',"And Hiram king of Tyre sent his servants unto Solomon; for he had heard that they had anointed him king in the room of his father: for Hiram was ever a lover of David."],
['1Kings','1Ki','05','002','2',"And Solomon sent to Hiram, saying,"],
['1Kings','1Ki','05','003','3',"Thou knowest how that David my father could not build an house unto the name of the LORD his God for the wars which were about him on every side, until the LORD put them under the soles of his feet."],
['1Kings','1Ki','05','004','4',"But now the LORD my God hath given me rest on every side, \emph{so} \emph{that} \emph{there} \emph{is} neither adversary nor evil occurrent."],
['1Kings','1Ki','05','005','5',"And, behold, I purpose to build an house unto the name of the LORD my God, as the LORD spake unto David my father, saying, Thy son, whom I will set upon thy throne in thy room, he shall build an house unto my name."],
['1Kings','1Ki','05','006','6',"Now therefore command thou that they hew me cedar trees out of Lebanon; and my servants shall be with thy servants: and unto thee will I give hire for thy servants according to all that thou shalt appoint: for thou knowest that \emph{there} \emph{is} not among us any that can skill to hew timber like unto the Sidonians."],
['1Kings','1Ki','05','007','7',"And it came to pass, when Hiram heard the words of Solomon, that he rejoiced greatly, and said, Blessed \emph{be} the LORD this day, which hath given unto David a wise son over this great people."],
['1Kings','1Ki','05','008','8',"And Hiram sent to Solomon, saying, I have considered the things which thou sentest to me for: \emph{and} I will do all thy desire concerning timber of cedar, and concerning timber of fir."],
['1Kings','1Ki','05','009','9',"My servants shall bring \emph{them} down from Lebanon unto the sea: and I will convey them by sea in floats unto the place that thou shalt appoint me, and will cause them to be discharged there, and thou shalt receive \emph{them}: and thou shalt accomplish my desire, in giving food for my household."],
['1Kings','1Ki','05','010','10',"So Hiram gave Solomon cedar trees and fir trees \emph{according} \emph{to} all his desire."],
['1Kings','1Ki','05','011','11',"And Solomon gave Hiram twenty thousand measures of wheat \emph{for} food to his household, and twenty measures of pure oil: thus gave Solomon to Hiram year by year."],
['1Kings','1Ki','05','012','12',"And the LORD gave Solomon wisdom, as he promised him: and there was peace between Hiram and Solomon; and they two made a league together."],
['1Kings','1Ki','05','013','13',"And king Solomon raised a levy out of all Israel; and the levy was thirty thousand men."],
['1Kings','1Ki','05','014','14',"And he sent them to Lebanon, ten thousand a month by courses: a month they were in Lebanon, \emph{and} two months at home: and Adoniram \emph{was} over the levy."],
['1Kings','1Ki','05','015','15',"And Solomon had threescore and ten thousand that bare burdens, and fourscore thousand hewers in the mountains;"],
['1Kings','1Ki','05','016','16',"Beside the chief of Solomon's officers which \emph{were} over the work, three thousand and three hundred, which ruled over the people that wrought in the work."],
['1Kings','1Ki','05','017','17',"And the king commanded, and they brought great stones, costly stones, \emph{and} hewed stones, to lay the foundation of the house."],
['1Kings','1Ki','05','018','18',"And Solomon's builders and Hiram's builders did hew \emph{them}, and the stonesquarers: so they prepared timber and stones to build the house."]
]

FirstKings06 = [
    ['1Kings','1Ki','06','001','1',"And it came to pass in the four hundred and eightieth year after the children of Israel were come out of the land of Egypt, in the fourth year of Solomon's reign over Israel, in the month Zif, which \emph{is} the second month, that he began to build the house of the LORD."],
['1Kings','1Ki','06','002','2',"And the house which king Solomon built for the LORD, the length thereof \emph{was} threescore cubits, and the breadth thereof twenty \emph{cubits}, and the height thereof thirty cubits."],
['1Kings','1Ki','06','003','3',"And the porch before the temple of the house, twenty cubits \emph{was} the length thereof, according to the breadth of the house; \emph{and} ten cubits \emph{was} the breadth thereof before the house."],
['1Kings','1Ki','06','004','4',"And for the house he made windows of narrow lights."],
['1Kings','1Ki','06','005','5',"And against the wall of the house he built chambers round about, \emph{against} the walls of the house round about, \emph{both} of the temple and of the oracle: and he made chambers round about:"],
['1Kings','1Ki','06','006','6',"The nethermost chamber \emph{was} five cubits broad, and the middle \emph{was} six cubits broad, and the third \emph{was} seven cubits broad: for without \emph{in} \emph{the} \emph{wall} of the house he made narrowed rests round about, that \emph{the} \emph{beams} should not be fastened in the walls of the house."],
['1Kings','1Ki','06','007','7',"And the house, when it was in building, was built of stone made ready before it was brought thither: so that there was neither hammer nor axe \emph{nor} any tool of iron heard in the house, while it was in building."],
['1Kings','1Ki','06','008','8',"The door for the middle chamber \emph{was} in the right side of the house: and they went up with winding stairs into the middle \emph{chamber}, and out of the middle into the third."],
['1Kings','1Ki','06','009','9',"So he built the house, and finished it; and covered the house with beams and boards of cedar."],
['1Kings','1Ki','06','010','10',"And \emph{then} he built chambers against all the house, five cubits high: and they rested on the house \emph{with} timber of cedar."],
['1Kings','1Ki','06','011','11',"And the word of the LORD came to Solomon, saying,"],
['1Kings','1Ki','06','012','12',"\emph{Concerning} this house which thou art in building, if thou wilt walk in my statutes, and execute my judgments, and keep all my commandments to walk in them; then will I perform my word with thee, which I spake unto David thy father:"],
['1Kings','1Ki','06','013','13',"And I will dwell among the children of Israel, and will not forsake my people Israel."],
['1Kings','1Ki','06','014','14',"So Solomon built the house, and finished it."],
['1Kings','1Ki','06','015','15',"And he built the walls of the house within with boards of cedar, both the floor of the house, and the walls of the cieling: \emph{and} he covered \emph{them} on the inside with wood, and covered the floor of the house with planks of fir."],
['1Kings','1Ki','06','016','16',"And he built twenty cubits on the sides of the house, both the floor and the walls with boards of cedar: he even built \emph{them} for it within, \emph{even} for the oracle, \emph{even} for the most holy \emph{place}."],
['1Kings','1Ki','06','017','17',"And the house, that \emph{is}, the temple before it, was forty cubits \emph{long}."],
['1Kings','1Ki','06','018','18',"And the cedar of the house within \emph{was} carved with knops and open flowers: all \emph{was} cedar; there was no stone seen."],
['1Kings','1Ki','06','019','19',"And the oracle he prepared in the house within, to set there the ark of the covenant of the LORD."],
['1Kings','1Ki','06','020','20',"And the oracle in the forepart \emph{was} twenty cubits in length, and twenty cubits in breadth, and twenty cubits in the height thereof: and he overlaid it with pure gold; and \emph{so} covered the altar \emph{which} \emph{was} \emph{of} cedar."],
['1Kings','1Ki','06','021','21',"So Solomon overlaid the house within with pure gold: and he made a partition by the chains of gold before the oracle; and he overlaid it with gold."],
['1Kings','1Ki','06','022','22',"And the whole house he overlaid with gold, until he had finished all the house: also the whole altar that \emph{was} by the oracle he overlaid with gold."],
['1Kings','1Ki','06','023','23',"And within the oracle he made two cherubims \emph{of} olive tree, \emph{each} ten cubits high."],
['1Kings','1Ki','06','024','24',"And five cubits \emph{was} the one wing of the cherub, and five cubits the other wing of the cherub: from the uttermost part of the one wing unto the uttermost part of the other \emph{were} ten cubits."],
['1Kings','1Ki','06','025','25',"And the other cherub \emph{was} ten cubits: both the cherubims \emph{were} of one measure and one size."],
['1Kings','1Ki','06','026','26',"The height of the one cherub \emph{was} ten cubits, and so \emph{was} \emph{it} of the other cherub."],
['1Kings','1Ki','06','027','27',"And he set the cherubims within the inner house: and they stretched forth the wings of the cherubims, so that the wing of the one touched the \emph{one} wall, and the wing of the other cherub touched the other wall; and their wings touched one another in the midst of the house."],
['1Kings','1Ki','06','028','28',"And he overlaid the cherubims with gold."],
['1Kings','1Ki','06','029','29',"And he carved all the walls of the house round about with carved figures of cherubims and palm trees and open flowers, within and without."],
['1Kings','1Ki','06','030','30',"And the floor of the house he overlaid with gold, within and without."],
['1Kings','1Ki','06','031','31',"And for the entering of the oracle he made doors \emph{of} olive tree: the lintel \emph{and} side posts \emph{were} a fifth part \emph{of} \emph{the} \emph{wall}."],
['1Kings','1Ki','06','032','32',"The two doors also \emph{were} \emph{of} olive tree; and he carved upon them carvings of cherubims and palm trees and open flowers, and overlaid \emph{them} with gold, and spread gold upon the cherubims, and upon the palm trees."],
['1Kings','1Ki','06','033','33',"So also made he for the door of the temple posts \emph{of} olive tree, a fourth part \emph{of} \emph{the} \emph{wall}."],
['1Kings','1Ki','06','034','34',"And the two doors \emph{were} \emph{of} fir tree: the two leaves of the one door \emph{were} folding, and the two leaves of the other door \emph{were} folding."],
['1Kings','1Ki','06','035','35',"And he carved \emph{thereon} cherubims and palm trees and open flowers: and covered \emph{them} with gold fitted upon the carved work."],
['1Kings','1Ki','06','036','36',"And he built the inner court with three rows of hewed stone, and a row of cedar beams."],
['1Kings','1Ki','06','037','37',"In the fourth year was the foundation of the house of the LORD laid, in the month Zif:"],
['1Kings','1Ki','06','038','38',"And in the eleventh year, in the month Bul, which \emph{is} the eighth month, was the house finished throughout all the parts thereof, and according to all the fashion of it. So was he seven years in building it."]
]

FirstKings07 = [
    ['1Kings','1Ki','07','001','1',"But Solomon was building his own house thirteen years, and he finished all his house."],
['1Kings','1Ki','07','002','2',"He built also the house of the forest of Lebanon; the length thereof \emph{was} an hundred cubits, and the breadth thereof fifty cubits, and the height thereof thirty cubits, upon four rows of cedar pillars, with cedar beams upon the pillars."],
['1Kings','1Ki','07','003','3',"And \emph{it} \emph{was} covered with cedar above upon the beams, that \emph{lay} on forty five pillars, fifteen \emph{in} a row."],
['1Kings','1Ki','07','004','4',"And \emph{there} \emph{were} windows \emph{in} three rows, and light \emph{was} against light \emph{in} three ranks."],
['1Kings','1Ki','07','005','5',"And all the doors and posts \emph{were} square, with the windows: and light \emph{was} against light \emph{in} three ranks."],
['1Kings','1Ki','07','006','6',"And he made a porch of pillars; the length thereof \emph{was} fifty cubits, and the breadth thereof thirty cubits: and the porch \emph{was} before them: and the \emph{other} pillars and the thick beam \emph{were} before them."],
['1Kings','1Ki','07','007','7',"Then he made a porch for the throne where he might judge, \emph{even} the porch of judgment: and \emph{it} \emph{was} covered with cedar from one side of the floor to the other."],
['1Kings','1Ki','07','008','8',"And his house where he dwelt \emph{had} another court within the porch, \emph{which} was of the like work. Solomon made also an house for Pharaoh's daughter, whom he had taken \emph{to} \emph{wife}, like unto this porch."],
['1Kings','1Ki','07','009','9',"All these \emph{were} \emph{of} costly stones, according to the measures of hewed stones, sawed with saws, within and without, even from the foundation unto the coping, and \emph{so} on the outside toward the great court."],
['1Kings','1Ki','07','010','10',"And the foundation \emph{was} \emph{of} costly stones, even great stones, stones of ten cubits, and stones of eight cubits."],
['1Kings','1Ki','07','011','11',"And above \emph{were} costly stones, after the measures of hewed stones, and cedars."],
['1Kings','1Ki','07','012','12',"And the great court round about \emph{was} with three rows of hewed stones, and a row of cedar beams, both for the inner court of the house of the LORD, and for the porch of the house."],
['1Kings','1Ki','07','013','13',"And king Solomon sent and fetched Hiram out of Tyre."],
['1Kings','1Ki','07','014','14',"He \emph{was} a widow's son of the tribe of Naphtali, and his father \emph{was} a man of Tyre, a worker in brass: and he was filled with wisdom, and understanding, and cunning to work all works in brass. And he came to king Solomon, and wrought all his work."],
['1Kings','1Ki','07','015','15',"For he cast two pillars of brass, of eighteen cubits high apiece: and a line of twelve cubits did compass either of them about."],
['1Kings','1Ki','07','016','16',"And he made two chapiters \emph{of} molten brass, to set upon the tops of the pillars: the height of the one chapiter \emph{was} five cubits, and the height of the other chapiter \emph{was} five cubits:"],
['1Kings','1Ki','07','017','17',"\emph{And} nets of checker work, and wreaths of chain work, for the chapiters which \emph{were} upon the top of the pillars; seven for the one chapiter, and seven for the other chapiter."],
['1Kings','1Ki','07','018','18',"And he made the pillars, and two rows round about upon the one network, to cover the chapiters that \emph{were} upon the top, with pomegranates: and so did he for the other chapiter."],
['1Kings','1Ki','07','019','19',"And the chapiters that \emph{were} upon the top of the pillars \emph{were} of lily work in the porch, four cubits."],
['1Kings','1Ki','07','020','20',"And the chapiters upon the two pillars \emph{had} \emph{pomegranates} also above, over against the belly which \emph{was} by the network: and the pomegranates \emph{were} two hundred in rows round about upon the other chapiter."],
['1Kings','1Ki','07','021','21',"And he set up the pillars in the porch of the temple: and he set up the right pillar, and called the name thereof Jachin: and he set up the left pillar, and called the name thereof Boaz."],
['1Kings','1Ki','07','022','22',"And upon the top of the pillars \emph{was} lily work: so was the work of the pillars finished."],
['1Kings','1Ki','07','023','23',"And he made a molten sea, ten cubits from the one brim to the other: \emph{it} \emph{was} round all about, and his height \emph{was} five cubits: and a line of thirty cubits did compass it round about."],
['1Kings','1Ki','07','024','24',"And under the brim of it round about \emph{there} \emph{were} knops compassing it, ten in a cubit, compassing the sea round about: the knops \emph{were} cast in two rows, when it was cast."],
['1Kings','1Ki','07','025','25',"It stood upon twelve oxen, three looking toward the north, and three looking toward the west, and three looking toward the south, and three looking toward the east: and the sea \emph{was} \emph{set} above upon them, and all their hinder parts \emph{were} inward."],
['1Kings','1Ki','07','026','26',"And it \emph{was} an hand breadth thick, and the brim thereof was wrought like the brim of a cup, with flowers of lilies: it contained two thousand baths."],
['1Kings','1Ki','07','027','27',"And he made ten bases of brass; four cubits \emph{was} the length of one base, and four cubits the breadth thereof, and three cubits the height of it."],
['1Kings','1Ki','07','028','28',"And the work of the bases \emph{was} on this \emph{manner}: they had borders, and the borders \emph{were} between the ledges:"],
['1Kings','1Ki','07','029','29',"And on the borders that \emph{were} between the ledges \emph{were} lions, oxen, and cherubims: and upon the ledges \emph{there} \emph{was} a base above: and beneath the lions and oxen \emph{were} certain additions made of thin work."],
['1Kings','1Ki','07','030','30',"And every base had four brasen wheels, and plates of brass: and the four corners thereof had undersetters: under the laver \emph{were} undersetters molten, at the side of every addition."],
['1Kings','1Ki','07','031','31',"And the mouth of it within the chapiter and above \emph{was} a cubit: but the mouth thereof \emph{was} round \emph{after} the work of the base, a cubit and an half: and also upon the mouth of it \emph{were} gravings with their borders, foursquare, not round."],
['1Kings','1Ki','07','032','32',"And under the borders \emph{were} four wheels; and the axletrees of the wheels \emph{were} \emph{joined} to the base: and the height of a wheel \emph{was} a cubit and half a cubit."],
['1Kings','1Ki','07','033','33',"And the work of the wheels \emph{was} like the work of a chariot wheel: their axletrees, and their naves, and their felloes, and their spokes, \emph{were} all molten."],
['1Kings','1Ki','07','034','34',"And \emph{there} \emph{were} four undersetters to the four corners of one base: \emph{and} the undersetters \emph{were} of the very base itself."],
['1Kings','1Ki','07','035','35',"And in the top of the base \emph{was} \emph{there} a round compass of half a cubit high: and on the top of the base the ledges thereof and the borders thereof \emph{were} of the same."],
['1Kings','1Ki','07','036','36',"For on the plates of the ledges thereof, and on the borders thereof, he graved cherubims, lions, and palm trees, according to the proportion of every one, and additions round about."],
['1Kings','1Ki','07','037','37',"After this \emph{manner} he made the ten bases: all of them had one casting, one measure, \emph{and} one size."],
['1Kings','1Ki','07','038','38',"Then made he ten lavers of brass: one laver contained forty baths: \emph{and} every laver was four cubits: \emph{and} upon every one of the ten bases one laver."],
['1Kings','1Ki','07','039','39',"And he put five bases on the right side of the house, and five on the left side of the house: and he set the sea on the right side of the house eastward over against the south."],
['1Kings','1Ki','07','040','40',"And Hiram made the lavers, and the shovels, and the basons. So Hiram made an end of doing all the work that he made king Solomon for the house of the LORD:"],
['1Kings','1Ki','07','041','41',"The two pillars, and the \emph{two} bowls of the chapiters that \emph{were} on the top of the two pillars; and the two networks, to cover the two bowls of the chapiters which \emph{were} upon the top of the pillars;"],
['1Kings','1Ki','07','042','42',"And four hundred pomegranates for the two networks, \emph{even} two rows of pomegranates for one network, to cover the two bowls of the chapiters that \emph{were} upon the pillars;"],
['1Kings','1Ki','07','043','43',"And the ten bases, and ten lavers on the bases;"],
['1Kings','1Ki','07','044','44',"And one sea, and twelve oxen under the sea;"],
['1Kings','1Ki','07','045','45',"And the pots, and the shovels, and the basons: and all these vessels, which Hiram made to king Solomon for the house of the LORD, \emph{were} \emph{of} bright brass."],
['1Kings','1Ki','07','046','46',"In the plain of Jordan did the king cast them, in the clay ground between Succoth and Zarthan."],
['1Kings','1Ki','07','047','47',"And Solomon left all the vessels \emph{unweighed}, because they were exceeding many: neither was the weight of the brass found out."],
['1Kings','1Ki','07','048','48',"And Solomon made all the vessels that \emph{pertained} unto the house of the LORD: the altar of gold, and the table of gold, whereupon the shewbread \emph{was},"],
['1Kings','1Ki','07','049','49',"And the candlesticks of pure gold, five on the right \emph{side}, and five on the left, before the oracle, with the flowers, and the lamps, and the tongs \emph{of} gold,"],
['1Kings','1Ki','07','050','50',"And the bowls, and the snuffers, and the basons, and the spoons, and the censers \emph{of} pure gold; and the hinges \emph{of} gold, \emph{both} for the doors of the inner house, the most holy \emph{place,} \emph{and} for the doors of the house, \emph{to} \emph{wit}, of the temple."],
['1Kings','1Ki','07','051','51',"So was ended all the work that king Solomon made for the house of the LORD. And Solomon brought in the things which David his father had dedicated; \emph{even} the silver, and the gold, and the vessels, did he put among the treasures of the house of the LORD."]
]

FirstKings08 = [
    ['1Kings','1Ki','08','001','1',"Then Solomon assembled the elders of Israel, and all the heads of the tribes, the chief of the fathers of the children of Israel, unto king Solomon in Jerusalem, that they might bring up the ark of the covenant of the LORD out of the city of David, which \emph{is} Zion."],
['1Kings','1Ki','08','002','2',"And all the men of Israel assembled themselves unto king Solomon at the feast in the month Ethanim, which \emph{is} the seventh month."],
['1Kings','1Ki','08','003','3',"And all the elders of Israel came, and the priests took up the ark."],
['1Kings','1Ki','08','004','4',"And they brought up the ark of the LORD, and the tabernacle of the congregation, and all the holy vessels that \emph{were} in the tabernacle, even those did the priests and the Levites bring up."],
['1Kings','1Ki','08','005','5',"And king Solomon, and all the congregation of Israel, that were assembled unto him, \emph{were} with him before the ark, sacrificing sheep and oxen, that could not be told nor numbered for multitude."],
['1Kings','1Ki','08','006','6',"And the priests brought in the ark of the covenant of the LORD unto his place, into the oracle of the house, to the most holy \emph{place,} \emph{even} under the wings of the cherubims."],
['1Kings','1Ki','08','007','7',"For the cherubims spread forth \emph{their} two wings over the place of the ark, and the cherubims covered the ark and the staves thereof above."],
['1Kings','1Ki','08','008','8',"And they drew out the staves, that the ends of the staves were seen out in the holy \emph{place} before the oracle, and they were not seen without: and there they are unto this day."],
['1Kings','1Ki','08','009','9',"\emph{There} \emph{was} nothing in the ark save the two tables of stone, which Moses put there at Horeb, when the LORD made \emph{a} \emph{covenant} with the children of Israel, when they came out of the land of Egypt."],
['1Kings','1Ki','08','010','10',"And it came to pass, when the priests were come out of the holy \emph{place}, that the cloud filled the house of the LORD,"],
['1Kings','1Ki','08','011','11',"So that the priests could not stand to minister because of the cloud: for the glory of the LORD had filled the house of the LORD."],
['1Kings','1Ki','08','012','12',"Then spake Solomon, The LORD said that he would dwell in the thick darkness."],
['1Kings','1Ki','08','013','13',"I have surely built thee an house to dwell in, a settled place for thee to abide in for ever."],
['1Kings','1Ki','08','014','14',"And the king turned his face about, and blessed all the congregation of Israel: (and all the congregation of Israel stood;)"],
['1Kings','1Ki','08','015','15',"And he said, Blessed \emph{be} the LORD God of Israel, which spake with his mouth unto David my father, and hath with his hand fulfilled \emph{it}, saying,"],
['1Kings','1Ki','08','016','16',"Since the day that I brought forth my people Israel out of Egypt, I chose no city out of all the tribes of Israel to build an house, that my name might be therein; but I chose David to be over my people Israel."],
['1Kings','1Ki','08','017','17',"And it was in the heart of David my father to build an house for the name of the LORD God of Israel."],
['1Kings','1Ki','08','018','18',"And the LORD said unto David my father, Whereas it was in thine heart to build an house unto my name, thou didst well that it was in thine heart."],
['1Kings','1Ki','08','019','19',"Nevertheless thou shalt not build the house; but thy son that shall come forth out of thy loins, he shall build the house unto my name."],
['1Kings','1Ki','08','020','20',"And the LORD hath performed his word that he spake, and I am risen up in the room of David my father, and sit on the throne of Israel, as the LORD promised, and have built an house for the name of the LORD God of Israel."],
['1Kings','1Ki','08','021','21',"And I have set there a place for the ark, wherein \emph{is} the covenant of the LORD, which he made with our fathers, when he brought them out of the land of Egypt."],
['1Kings','1Ki','08','022','22',"And Solomon stood before the altar of the LORD in the presence of all the congregation of Israel, and spread forth his hands toward heaven:"],
['1Kings','1Ki','08','023','23',"And he said, LORD God of Israel, \emph{there} \emph{is} no God like thee, in heaven above, or on earth beneath, who keepest covenant and mercy with thy servants that walk before thee with all their heart:"],
['1Kings','1Ki','08','024','24',"Who hast kept with thy servant David my father that thou promisedst him: thou spakest also with thy mouth, and hast fulfilled \emph{it} with thine hand, as \emph{it} \emph{is} this day."],
['1Kings','1Ki','08','025','25',"Therefore now, LORD God of Israel, keep with thy servant David my father that thou promisedst him, saying, There shall not fail thee a man in my sight to sit on the throne of Israel; so that thy children take heed to their way, that they walk before me as thou hast walked before me."],
['1Kings','1Ki','08','026','26',"And now, O God of Israel, let thy word, I pray thee, be verified, which thou spakest unto thy servant David my father."],
['1Kings','1Ki','08','027','27',"But will God indeed dwell on the earth? behold, the heaven and heaven of heavens cannot contain thee; how much less this house that I have builded?"],
['1Kings','1Ki','08','028','28',"Yet have thou respect unto the prayer of thy servant, and to his supplication, O LORD my God, to hearken unto the cry and to the prayer, which thy servant prayeth before thee to day:"],
['1Kings','1Ki','08','029','29',"That thine eyes may be open toward this house night and day, \emph{even} toward the place of which thou hast said, My name shall be there: that thou mayest hearken unto the prayer which thy servant shall make toward this place."],
['1Kings','1Ki','08','030','30',"And hearken thou to the supplication of thy servant, and of thy people Israel, when they shall pray toward this place: and hear thou in heaven thy dwelling place: and when thou hearest, forgive."],
['1Kings','1Ki','08','031','31',"If any man trespass against his neighbour, and an oath be laid upon him to cause him to swear, and the oath come before thine altar in this house:"],
['1Kings','1Ki','08','032','32',"Then hear thou in heaven, and do, and judge thy servants, condemning the wicked, to bring his way upon his head; and justifying the righteous, to give him according to his righteousness."],
['1Kings','1Ki','08','033','33',"When thy people Israel be smitten down before the enemy, because they have sinned against thee, and shall turn again to thee, and confess thy name, and pray, and make supplication unto thee in this house:"],
['1Kings','1Ki','08','034','34',"Then hear thou in heaven, and forgive the sin of thy people Israel, and bring them again unto the land which thou gavest unto their fathers."],
['1Kings','1Ki','08','035','35',"When heaven is shut up, and there is no rain, because they have sinned against thee; if they pray toward this place, and confess thy name, and turn from their sin, when thou afflictest them:"],
['1Kings','1Ki','08','036','36',"Then hear thou in heaven, and forgive the sin of thy servants, and of thy people Israel, that thou teach them the good way wherein they should walk, and give rain upon thy land, which thou hast given to thy people for an inheritance."],
['1Kings','1Ki','08','037','37',"If there be in the land famine, if there be pestilence, blasting, mildew, locust, \emph{or} if there be caterpiller; if their enemy besiege them in the land of their cities; whatsoever plague, whatsoever sickness \emph{there} \emph{be};"],
['1Kings','1Ki','08','038','38',"What prayer and supplication soever be \emph{made} by any man, \emph{or} by all thy people Israel, which shall know every man the plague of his own heart, and spread forth his hands toward this house:"],
['1Kings','1Ki','08','039','39',"Then hear thou in heaven thy dwelling place, and forgive, and do, and give to every man according to his ways, whose heart thou knowest; (for thou, \emph{even} thou only, knowest the hearts of all the children of men;)"],
['1Kings','1Ki','08','040','40',"That they may fear thee all the days that they live in the land which thou gavest unto our fathers."],
['1Kings','1Ki','08','041','41',"Moreover concerning a stranger, that \emph{is} not of thy people Israel, but cometh out of a far country for thy name's sake;"],
['1Kings','1Ki','08','042','42',"(For they shall hear of thy great name, and of thy strong hand, and of thy stretched out arm;) when he shall come and pray toward this house;"],
['1Kings','1Ki','08','043','43',"Hear thou in heaven thy dwelling place, and do according to all that the stranger calleth to thee for: that all people of the earth may know thy name, to fear thee, as \emph{do} thy people Israel; and that they may know that this house, which I have builded, is called by thy name."],
['1Kings','1Ki','08','044','44',"If thy people go out to battle against their enemy, whithersoever thou shalt send them, and shall pray unto the LORD toward the city which thou hast chosen, and \emph{toward} the house that I have built for thy name:"],
['1Kings','1Ki','08','045','45',"Then hear thou in heaven their prayer and their supplication, and maintain their cause."],
['1Kings','1Ki','08','046','46',"If they sin against thee, (for \emph{there} \emph{is} no man that sinneth not,) and thou be angry with them, and deliver them to the enemy, so that they carry them away captives unto the land of the enemy, far or near;"],
['1Kings','1Ki','08','047','47',"\emph{Yet} if they shall bethink themselves in the land whither they were carried captives, and repent, and make supplication unto thee in the land of them that carried them captives, saying, We have sinned, and have done perversely, we have committed wickedness;"],
['1Kings','1Ki','08','048','48',"And \emph{so} return unto thee with all their heart, and with all their soul, in the land of their enemies, which led them away captive, and pray unto thee toward their land, which thou gavest unto their fathers, the city which thou hast chosen, and the house which I have built for thy name:"],
['1Kings','1Ki','08','049','49',"Then hear thou their prayer and their supplication in heaven thy dwelling place, and maintain their cause,"],
['1Kings','1Ki','08','050','50',"And forgive thy people that have sinned against thee, and all their transgressions wherein they have transgressed against thee, and give them compassion before them who carried them captive, that they may have compassion on them:"],
['1Kings','1Ki','08','051','51',"For they \emph{be} thy people, and thine inheritance, which thou broughtest forth out of Egypt, from the midst of the furnace of iron:"],
['1Kings','1Ki','08','052','52',"That thine eyes may be open unto the supplication of thy servant, and unto the supplication of thy people Israel, to hearken unto them in all that they call for unto thee."],
['1Kings','1Ki','08','053','53',"For thou didst separate them from among all the people of the earth, \emph{to} \emph{be} thine inheritance, as thou spakest by the hand of Moses thy servant, when thou broughtest our fathers out of Egypt, O Lord GOD."],
['1Kings','1Ki','08','054','54',"And it was \emph{so}, that when Solomon had made an end of praying all this prayer and supplication unto the LORD, he arose from before the altar of the LORD, from kneeling on his knees with his hands spread up to heaven."],
['1Kings','1Ki','08','055','55',"And he stood, and blessed all the congregation of Israel with a loud voice, saying,"],
['1Kings','1Ki','08','056','56',"Blessed \emph{be} the LORD, that hath given rest unto his people Israel, according to all that he promised: there hath not failed one word of all his good promise, which he promised by the hand of Moses his servant."],
['1Kings','1Ki','08','057','57',"The LORD our God be with us, as he was with our fathers: let him not leave us, nor forsake us:"],
['1Kings','1Ki','08','058','58',"That he may incline our hearts unto him, to walk in all his ways, and to keep his commandments, and his statutes, and his judgments, which he commanded our fathers."],
['1Kings','1Ki','08','059','59',"And let these my words, wherewith I have made supplication before the LORD, be nigh unto the LORD our God day and night, that he maintain the cause of his servant, and the cause of his people Israel at all times, as the matter shall require:"],
['1Kings','1Ki','08','060','60',"That all the people of the earth may know that the LORD \emph{is} God, \emph{and} \emph{that} \emph{there} \emph{is} none else."],
['1Kings','1Ki','08','061','61',"Let your heart therefore be perfect with the LORD our God, to walk in his statutes, and to keep his commandments, as at this day."],
['1Kings','1Ki','08','062','62',"And the king, and all Israel with him, offered sacrifice before the LORD."],
['1Kings','1Ki','08','063','63',"And Solomon offered a sacrifice of peace offerings, which he offered unto the LORD, two and twenty thousand oxen, and an hundred and twenty thousand sheep. So the king and all the children of Israel dedicated the house of the LORD."],
['1Kings','1Ki','08','064','64',"The same day did the king hallow the middle of the court that \emph{was} before the house of the LORD: for there he offered burnt offerings, and meat offerings, and the fat of the peace offerings: because the brasen altar that \emph{was} before the LORD \emph{was} too little to receive the burnt offerings, and meat offerings, and the fat of the peace offerings."],
['1Kings','1Ki','08','065','65',"And at that time Solomon held a feast, and all Israel with him, a great congregation, from the entering in of Hamath unto the river of Egypt, before the LORD our God, seven days and seven days, \emph{even} fourteen days."],
['1Kings','1Ki','08','066','66',"On the eighth day he sent the people away: and they blessed the king, and went unto their tents joyful and glad of heart for all the goodness that the LORD had done for David his servant, and for Israel his people."]
]

FirstKings09 = [
    ['1Kings','1Ki','09','001','1',"And it came to pass, when Solomon had finished the building of the house of the LORD, and the king's house, and all Solomon's desire which he was pleased to do,"],
['1Kings','1Ki','09','002','2',"That the LORD appeared to Solomon the second time, as he had appeared unto him at Gibeon."],
['1Kings','1Ki','09','003','3',"And the LORD said unto him, I have heard thy prayer and thy supplication, that thou hast made before me: I have hallowed this house, which thou hast built, to put my name there for ever; and mine eyes and mine heart shall be there perpetually."],
['1Kings','1Ki','09','004','4',"And if thou wilt walk before me, as David thy father walked, in integrity of heart, and in uprightness, to do according to all that I have commanded thee, \emph{and} wilt keep my statutes and my judgments:"],
['1Kings','1Ki','09','005','5',"Then I will establish the throne of thy kingdom upon Israel for ever, as I promised to David thy father, saying, There shall not fail thee a man upon the throne of Israel."],
['1Kings','1Ki','09','006','6',"\emph{But} if ye shall at all turn from following me, ye or your children, and will not keep my commandments \emph{and} my statutes which I have set before you, but go and serve other gods, and worship them:"],
['1Kings','1Ki','09','007','7',"Then will I cut off Israel out of the land which I have given them; and this house, which I have hallowed for my name, will I cast out of my sight; and Israel shall be a proverb and a byword among all people:"],
['1Kings','1Ki','09','008','8',"And at this house, \emph{which} is high, every one that passeth by it shall be astonished, and shall hiss; and they shall say, Why hath the LORD done thus unto this land, and to this house?"],
['1Kings','1Ki','09','009','9',"And they shall answer, Because they forsook the LORD their God, who brought forth their fathers out of the land of Egypt, and have taken hold upon other gods, and have worshipped them, and served them: therefore hath the LORD brought upon them all this evil."],
['1Kings','1Ki','09','010','10',"And it came to pass at the end of twenty years, when Solomon had built the two houses, the house of the LORD, and the king's house,"],
['1Kings','1Ki','09','011','11',"\emph{(Now} Hiram the king of Tyre had furnished Solomon with cedar trees and fir trees, and with gold, according to all his desire,) that then king Solomon gave Hiram twenty cities in the land of Galilee."],
['1Kings','1Ki','09','012','12',"And Hiram came out from Tyre to see the cities which Solomon had given him; and they pleased him not."],
['1Kings','1Ki','09','013','13',"And he said, What cities \emph{are} these which thou hast given me, my brother? And he called them the land of Cabul unto this day."],
['1Kings','1Ki','09','014','14',"And Hiram sent to the king sixscore talents of gold."],
['1Kings','1Ki','09','015','15',"And this \emph{is} the reason of the levy which king Solomon raised; for to build the house of the LORD, and his own house, and Millo, and the wall of Jerusalem, and Hazor, and Megiddo, and Gezer."],
['1Kings','1Ki','09','016','16',"\emph{For} Pharaoh king of Egypt had gone up, and taken Gezer, and burnt it with fire, and slain the Canaanites that dwelt in the city, and given it \emph{for} a present unto his daughter, Solomon's wife."],
['1Kings','1Ki','09','017','17',"And Solomon built Gezer, and Beth-horon the nether,"],
['1Kings','1Ki','09','018','18',"And Baalath, and Tadmor in the wilderness, in the land,"],
['1Kings','1Ki','09','019','19',"And all the cities of store that Solomon had, and cities for his chariots, and cities for his horsemen, and that which Solomon desired to build in Jerusalem, and in Lebanon, and in all the land of his dominion."],
['1Kings','1Ki','09','020','20',"\emph{And} all the people \emph{that} \emph{were} left of the Amorites, Hittites, Perizzites, Hivites, and Jebusites, which \emph{were} not of the children of Israel,"],
['1Kings','1Ki','09','021','21',"Their children that were left after them in the land, whom the children of Israel also were not able utterly to destroy, upon those did Solomon levy a tribute of bondservice unto this day."],
['1Kings','1Ki','09','022','22',"But of the children of Israel did Solomon make no bondmen: but they \emph{were} men of war, and his servants, and his princes, and his captains, and rulers of his chariots, and his horsemen."],
['1Kings','1Ki','09','023','23',"These \emph{were} the chief of the officers that \emph{were} over Solomon's work, five hundred and fifty, which bare rule over the people that wrought in the work."],
['1Kings','1Ki','09','024','24',"But Pharaoh's daughter came up out of the city of David unto her house which \emph{Solomon} had built for her: then did he build Millo."],
['1Kings','1Ki','09','025','25',"And three times in a year did Solomon offer burnt offerings and peace offerings upon the altar which he built unto the LORD, and he burnt incense upon the altar that \emph{was} before the LORD. So he finished the house."],
['1Kings','1Ki','09','026','26',"And king Solomon made a navy of ships in Ezion-geber, which \emph{is} beside Eloth, on the shore of the Red sea, in the land of Edom."],
['1Kings','1Ki','09','027','27',"And Hiram sent in the navy his servants, shipmen that had knowledge of the sea, with the servants of Solomon."],
['1Kings','1Ki','09','028','28',"And they came to Ophir, and fetched from thence gold, four hundred and twenty talents, and brought \emph{it} to king Solomon."]
]

FirstKings10 = [
['1Kings','1Ki','10','001','1',"And when the queen of Sheba heard of the fame of Solomon concerning the name of the LORD, she came to prove him with hard questions."],
['1Kings','1Ki','10','002','2',"And she came to Jerusalem with a very great train, with camels that bare spices, and very much gold, and precious stones: and when she was come to Solomon, she communed with him of all that was in her heart."],
['1Kings','1Ki','10','003','3',"And Solomon told her all her questions: there was not \emph{any} thing hid from the king, which he told her not."],
['1Kings','1Ki','10','004','4',"And when the queen of Sheba had seen all Solomon's wisdom, and the house that he had built,"],
['1Kings','1Ki','10','005','5',"And the meat of his table, and the sitting of his servants, and the attendance of his ministers, and their apparel, and his cupbearers, and his ascent by which he went up unto the house of the LORD; there was no more spirit in her."],
['1Kings','1Ki','10','006','6',"And she said to the king, It was a true report that I heard in mine own land of thy acts and of thy wisdom."],
['1Kings','1Ki','10','007','7',"Howbeit I believed not the words, until I came, and mine eyes had seen \emph{it}: and, behold, the half was not told me: thy wisdom and prosperity exceedeth the fame which I heard."],
['1Kings','1Ki','10','008','8',"Happy \emph{are} thy men, happy \emph{are} these thy servants, which stand continually before thee, \emph{and} that hear thy wisdom."],
['1Kings','1Ki','10','009','9',"Blessed be the LORD thy God, which delighted in thee, to set thee on the throne of Israel: because the LORD loved Israel for ever, therefore made he thee king, to do judgment and justice."],
['1Kings','1Ki','10','010','10',"And she gave the king an hundred and twenty talents of gold, and of spices very great store, and precious stones: there came no more such abundance of spices as these which the queen of Sheba gave to king Solomon."],
['1Kings','1Ki','10','011','11',"And the navy also of Hiram, that brought gold from Ophir, brought in from Ophir great plenty of almug trees, and precious stones."],
['1Kings','1Ki','10','012','12',"And the king made of the almug trees pillars for the house of the LORD, and for the king's house, harps also and psalteries for singers: there came no such almug trees, nor were seen unto this day."],
['1Kings','1Ki','10','013','13',"And king Solomon gave unto the queen of Sheba all her desire, whatsoever she asked, beside \emph{that} which Solomon gave her of his royal bounty. So she turned and went to her own country, she and her servants."],
['1Kings','1Ki','10','014','14',"Now the weight of gold that came to Solomon in one year was six hundred threescore and six talents of gold,"],
['1Kings','1Ki','10','015','15',"Beside \emph{that} \emph{he} \emph{had} of the merchantmen, and of the traffick of the spice merchants, and of all the kings of Arabia, and of the governors of the country."],
['1Kings','1Ki','10','016','16',"And king Solomon made two hundred targets \emph{of} beaten gold: six hundred \emph{shekels} of gold went to one target."],
['1Kings','1Ki','10','017','17',"And \emph{he} \emph{made} three hundred shields \emph{of} beaten gold; three pound of gold went to one shield: and the king put them in the house of the forest of Lebanon."],
['1Kings','1Ki','10','018','18',"Moreover the king made a great throne of ivory, and overlaid it with the best gold."],
['1Kings','1Ki','10','019','19',"The throne had six steps, and the top of the throne \emph{was} round behind: and \emph{there} \emph{were} stays on either side on the place of the seat, and two lions stood beside the stays."],
['1Kings','1Ki','10','020','20',"And twelve lions stood there on the one side and on the other upon the six steps: there was not the like made in any kingdom."],
['1Kings','1Ki','10','021','21',"And all king Solomon's drinking vessels \emph{were} \emph{of} gold, and all the vessels of the house of the forest of Lebanon \emph{were} \emph{of} pure gold; none \emph{were} \emph{of} silver: it was nothing accounted of in the days of Solomon."],
['1Kings','1Ki','10','022','22',"For the king had at sea a navy of Tharshish with the navy of Hiram: once in three years came the navy of Tharshish, bringing gold, and silver, ivory, and apes, and peacocks."],
['1Kings','1Ki','10','023','23',"So king Solomon exceeded all the kings of the earth for riches and for wisdom."],
['1Kings','1Ki','10','024','24',"And all the earth sought to Solomon, to hear his wisdom, which God had put in his heart."],
['1Kings','1Ki','10','025','25',"And they brought every man his present, vessels of silver, and vessels of gold, and garments, and armour, and spices, horses, and mules, a rate year by year."],
['1Kings','1Ki','10','026','26',"And Solomon gathered together chariots and horsemen: and he had a thousand and four hundred chariots, and twelve thousand horsemen, whom he bestowed in the cities for chariots, and with the king at Jerusalem."],
['1Kings','1Ki','10','027','27',"And the king made silver \emph{to} \emph{be} in Jerusalem as stones, and cedars made he \emph{to} \emph{be} as the sycomore trees that \emph{are} in the vale, for abundance."],
['1Kings','1Ki','10','028','28',"And Solomon had horses brought out of Egypt, and linen yarn: the king's merchants received the linen yarn at a price."],
['1Kings','1Ki','10','029','29',"And a chariot came up and went out of Egypt for six hundred \emph{shekels} of silver, and an horse for an hundred and fifty: and so for all the kings of the Hittites, and for the kings of Syria, did they bring \emph{them} out by their means."]
]

FirstKings11 = [
['1Kings','1Ki','11','001','1',"But king Solomon loved many strange women, together with the daughter of Pharaoh, women of the Moabites, Ammonites, Edomites, Zidonians, \emph{and} Hittites;"],
['1Kings','1Ki','11','002','2',"Of the nations \emph{concerning} which the LORD said unto the children of Israel, Ye shall not go in to them, neither shall they come in unto you: \emph{for} surely they will turn away your heart after their gods: Solomon clave unto these in love."],
['1Kings','1Ki','11','003','3',"And he had seven hundred wives, princesses, and three hundred concubines: and his wives turned away his heart."],
['1Kings','1Ki','11','004','4',"For it came to pass, when Solomon was old, \emph{that} his wives turned away his heart after other gods: and his heart was not perfect with the LORD his God, as \emph{was} the heart of David his father."],
['1Kings','1Ki','11','005','5',"For Solomon went after Ashtoreth the goddess of the Zidonians, and after Milcom the abomination of the Ammonites."],
['1Kings','1Ki','11','006','6',"And Solomon did evil in the sight of the LORD, and went not fully after the LORD, as \emph{did} David his father."],
['1Kings','1Ki','11','007','7',"Then did Solomon build an high place for Chemosh, the abomination of Moab, in the hill that \emph{is} before Jerusalem, and for Molech, the abomination of the children of Ammon."],
['1Kings','1Ki','11','008','8',"And likewise did he for all his strange wives, which burnt incense and sacrificed unto their gods."],
['1Kings','1Ki','11','009','9',"And the LORD was angry with Solomon, because his heart was turned from the LORD God of Israel, which had appeared unto him twice,"],
['1Kings','1Ki','11','010','10',"And had commanded him concerning this thing, that he should not go after other gods: but he kept not that which the LORD commanded."],
['1Kings','1Ki','11','011','11',"Wherefore the LORD said unto Solomon, Forasmuch as this is done of thee, and thou hast not kept my covenant and my statutes, which I have commanded thee, I will surely rend the kingdom from thee, and will give it to thy servant."],
['1Kings','1Ki','11','012','12',"Notwithstanding in thy days I will not do it for David thy father's sake: \emph{but} I will rend it out of the hand of thy son."],
['1Kings','1Ki','11','013','13',"Howbeit I will not rend away all the kingdom; \emph{but} will give one tribe to thy son for David my servant's sake, and for Jerusalem's sake which I have chosen."],
['1Kings','1Ki','11','014','14',"And the LORD stirred up an adversary unto Solomon, Hadad the Edomite: he \emph{was} of the king's seed in Edom."],
['1Kings','1Ki','11','015','15',"For it came to pass, when David was in Edom, and Joab the captain of the host was gone up to bury the slain, after he had smitten every male in Edom;"],
['1Kings','1Ki','11','016','16',"(For six months did Joab remain there with all Israel, until he had cut off every male in Edom:)"],
['1Kings','1Ki','11','017','17',"That Hadad fled, he and certain Edomites of his father's servants with him, to go into Egypt; Hadad \emph{being} yet a little child."],
['1Kings','1Ki','11','018','18',"And they arose out of Midian, and came to Paran: and they took men with them out of Paran, and they came to Egypt, unto Pharaoh king of Egypt; which gave him an house, and appointed him victuals, and gave him land."],
['1Kings','1Ki','11','019','19',"And Hadad found great favour in the sight of Pharaoh, so that he gave him to wife the sister of his own wife, the sister of Tahpenes the queen."],
['1Kings','1Ki','11','020','20',"And the sister of Tahpenes bare him Genubath his son, whom Tahpenes weaned in Pharaoh's house: and Genubath was in Pharaoh's household among the sons of Pharaoh."],
['1Kings','1Ki','11','021','21',"And when Hadad heard in Egypt that David slept with his fathers, and that Joab the captain of the host was dead, Hadad said to Pharaoh, Let me depart, that I may go to mine own country."],
    ['1Kings','1Ki','11','022','22',"Then Pharaoh said unto him, But what hast thou lacked with me, that, behold, thou seekest to go to thine own country? And he answered, Nothing: howbeit let me go in any wise."],
    ['1Kings','1Ki','11','023','23',"And God stirred him up \emph{another} adversary, Rezon the son of Eliadah, which fled from his lord Hadadezer king of Zobah:"],
    ['1Kings','1Ki','11','024','24',"And he gathered men unto him, and became captain over a band, when David slew them \emph{of} \emph{Zobah}: and they went to Damascus, and dwelt therein, and reigned in Damascus."],
    ['1Kings','1Ki','11','025','25',"And he was an adversary to Israel all the days of Solomon, beside the mischief that Hadad \emph{did}: and he abhorred Israel, and reigned over Syria."],
    ['1Kings','1Ki','11','026','26',"And Jeroboam the son of Nebat, an Ephrathite of Zereda, Solomon's servant, whose mother's name \emph{was} Zeruah, a widow woman, even he lifted up \emph{his} hand against the king."],
    ['1Kings','1Ki','11','027','27',"And this \emph{was} the cause that he lifted up \emph{his} hand against the king: Solomon built Millo, \emph{and} repaired the breaches of the city of David his father."],
    ['1Kings','1Ki','11','028','28',"And the man Jeroboam \emph{was} a mighty man of valour: and Solomon seeing the young man that he was industrious, he made him ruler over all the charge of the house of Joseph."],
    ['1Kings','1Ki','11','029','29',"And it came to pass at that time when Jeroboam went out of Jerusalem, that the prophet Ahijah the Shilonite found him in the way; and he had clad himself with a new garment; and they two \emph{were} alone in the field:"],
    ['1Kings','1Ki','11','030','30',"And Ahijah caught the new garment that \emph{was} on him, and rent it \emph{in} twelve pieces:"],
['1Kings','1Ki','11','031','31',"And he said to Jeroboam, Take thee ten pieces: for thus saith the LORD, the God of Israel, Behold, I will rend the kingdom out of the hand of Solomon, and will give ten tribes to thee:"],
['1Kings','1Ki','11','032','32',"(But he shall have one tribe for my servant David's sake, and for Jerusalem's sake, the city which I have chosen out of all the tribes of Israel:)"],
['1Kings','1Ki','11','033','33',"Because that they have forsaken me, and have worshipped Ashtoreth the goddess of the Zidonians, Chemosh the god of the Moabites, and Milcom the god of the children of Ammon, and have not walked in my ways, to do \emph{that} \emph{which} \emph{is} right in mine eyes, and \emph{to} \emph{keep} my statutes and my judgments, as \emph{did} David his father."],
['1Kings','1Ki','11','034','34',"Howbeit I will not take the whole kingdom out of his hand: but I will make him prince all the days of his life for David my servant's sake, whom I chose, because he kept my commandments and my statutes:"],
['1Kings','1Ki','11','035','35',"But I will take the kingdom out of his son's hand, and will give it unto thee, \emph{even} ten tribes."],
['1Kings','1Ki','11','036','36',"And unto his son will I give one tribe, that David my servant may have a light alway before me in Jerusalem, the city which I have chosen me to put my name there."],
['1Kings','1Ki','11','037','37',"And I will take thee, and thou shalt reign according to all that thy soul desireth, and shalt be king over Israel."],
['1Kings','1Ki','11','038','38',"And it shall be, if thou wilt hearken unto all that I command thee, and wilt walk in my ways, and do \emph{that} \emph{is} right in my sight, to keep my statutes and my commandments, as David my servant did; that I will be with thee, and build thee a sure house, as I built for David, and will give Israel unto thee."],
    ['1Kings','1Ki','11','039','39',"And I will for this afflict the seed of David, but not for ever."],
    ['1Kings','1Ki','11','040','40',"Solomon sought therefore to kill Jeroboam. And Jeroboam arose, and fled into Egypt, unto Shishak king of Egypt, and was in Egypt until the death of Solomon."],
    ['1Kings','1Ki','11','041','41',"And the rest of the acts of Solomon, and all that he did, and his wisdom, \emph{are} they not written in the book of the acts of Solomon?"],
    ['1Kings','1Ki','11','042','42',"And the time that Solomon reigned in Jerusalem over all Israel \emph{was} forty years."],
    ['1Kings','1Ki','11','043','43',"And Solomon slept with his fathers, and was buried in the city of David his father: and Rehoboam his son reigned in his stead."]
]

FirstKings12 = [
    ['1Kings','1Ki','12','001','1',"And Rehoboam went to Shechem: for all Israel were come to Shechem to make him king."],
['1Kings','1Ki','12','002','2',"And it came to pass, when Jeroboam the son of Nebat, who was yet in Egypt, heard \emph{of} \emph{it}, (for he was fled from the presence of king Solomon, and Jeroboam dwelt in Egypt;)"],
['1Kings','1Ki','12','003','3',"That they sent and called him. And Jeroboam and all the congregation of Israel came, and spake unto Rehoboam, saying,"],
['1Kings','1Ki','12','004','4',"Thy father made our yoke grievous: now therefore make thou the grievous service of thy father, and his heavy yoke which he put upon us, lighter, and we will serve thee."],
['1Kings','1Ki','12','005','5',"And he said unto them, Depart yet \emph{for} three days, then come again to me. And the people departed."],
['1Kings','1Ki','12','006','6',"And king Rehoboam consulted with the old men, that stood before Solomon his father while he yet lived, and said, How do ye advise that I may answer this people?"],
['1Kings','1Ki','12','007','7',"And they spake unto him, saying, If thou wilt be a servant unto this people this day, and wilt serve them, and answer them, and speak good words to them, then they will be thy servants for ever."],
['1Kings','1Ki','12','008','8',"But he forsook the counsel of the old men, which they had given him, and consulted with the young men that were grown up with him, \emph{and} which stood before him:"],
['1Kings','1Ki','12','009','9',"And he said unto them, What counsel give ye that we may answer this people, who have spoken to me, saying, Make the yoke which thy father did put upon us lighter?"],
['1Kings','1Ki','12','010','10',"And the young men that were grown up with him spake unto him, saying, Thus shalt thou speak unto this people that spake unto thee, saying, Thy father made our yoke heavy, but make thou \emph{it} lighter unto us; thus shalt thou say unto them, My little \emph{finger} shall be thicker than my father's loins."],
['1Kings','1Ki','12','011','11',"And now whereas my father did lade you with a heavy yoke, I will add to your yoke: my father hath chastised you with whips, but I will chastise you with scorpions."],
['1Kings','1Ki','12','012','12',"So Jeroboam and all the people came to Rehoboam the third day, as the king had appointed, saying, Come to me again the third day."],
['1Kings','1Ki','12','013','13',"And the king answered the people roughly, and forsook the old men's counsel that they gave him;"],
['1Kings','1Ki','12','014','14',"And spake to them after the counsel of the young men, saying, My father made your yoke heavy, and I will add to your yoke: my father \emph{also} chastised you with whips, but I will chastise you with scorpions."],
['1Kings','1Ki','12','015','15',"Wherefore the king hearkened not unto the people; for the cause was from the LORD, that he might perform his saying, which the LORD spake by Ahijah the Shilonite unto Jeroboam the son of Nebat."],
['1Kings','1Ki','12','016','16',"So when all Israel saw that the king hearkened not unto them, the people answered the king, saying, What portion have we in David? neither \emph{have} \emph{we} inheritance in the son of Jesse: to your tents, O Israel: now see to thine own house, David. So Israel departed unto their tents."],
['1Kings','1Ki','12','017','17',"But \emph{as} \emph{for} the children of Israel which dwelt in the cities of Judah, Rehoboam reigned over them."],
['1Kings','1Ki','12','018','18',"Then king Rehoboam sent Adoram, who \emph{was} over the tribute; and all Israel stoned him with stones, that he died. Therefore king Rehoboam made speed to get him up to his chariot, to flee to Jerusalem."],
['1Kings','1Ki','12','019','19',"So Israel rebelled against the house of David unto this day."],
['1Kings','1Ki','12','020','20',"And it came to pass, when all Israel heard that Jeroboam was come again, that they sent and called him unto the congregation, and made him king over all Israel: there was none that followed the house of David, but the tribe of Judah only."],
['1Kings','1Ki','12','021','21',"And when Rehoboam was come to Jerusalem, he assembled all the house of Judah, with the tribe of Benjamin, an hundred and fourscore thousand chosen men, which were warriors, to fight against the house of Israel, to bring the kingdom again to Rehoboam the son of Solomon."],
['1Kings','1Ki','12','022','22',"But the word of God came unto Shemaiah the man of God, saying,"],
['1Kings','1Ki','12','023','23',"Speak unto Rehoboam, the son of Solomon, king of Judah, and unto all the house of Judah and Benjamin, and to the remnant of the people, saying,"],
['1Kings','1Ki','12','024','24',"Thus saith the LORD, Ye shall not go up, nor fight against your brethren the children of Israel: return every man to his house; for this thing is from me. They hearkened therefore to the word of the LORD, and returned to depart, according to the word of the LORD."],
['1Kings','1Ki','12','025','25',"Then Jeroboam built Shechem in mount Ephraim, and dwelt therein; and went out from thence, and built Penuel."],
['1Kings','1Ki','12','026','26',"And Jeroboam said in his heart, Now shall the kingdom return to the house of David:"],
['1Kings','1Ki','12','027','27',"If this people go up to do sacrifice in the house of the LORD at Jerusalem, then shall the heart of this people turn again unto their lord, \emph{even} unto Rehoboam king of Judah, and they shall kill me, and go again to Rehoboam king of Judah."],
['1Kings','1Ki','12','028','28',"Whereupon the king took counsel, and made two calves \emph{of} gold, and said unto them, It is too much for you to go up to Jerusalem: behold thy gods, O Israel, which brought thee up out of the land of Egypt."],
['1Kings','1Ki','12','029','29',"And he set the one in Beth-el, and the other put he in Dan."],
['1Kings','1Ki','12','030','30',"And this thing became a sin: for the people went \emph{to} \emph{worship} before the one, \emph{even} unto Dan."],
['1Kings','1Ki','12','031','31',"And he made an house of high places, and made priests of the lowest of the people, which were not of the sons of Levi."],
['1Kings','1Ki','12','032','32',"And Jeroboam ordained a feast in the eighth month, on the fifteenth day of the month, like unto the feast that \emph{is} in Judah, and he offered upon the altar. So did he in Beth-el, sacrificing unto the calves that he had made: and he placed in Beth-el the priests of the high places which he had made."],
['1Kings','1Ki','12','033','33',"So he offered upon the altar which he had made in Beth-el the fifteenth day of the eighth month, \emph{even} in the month which he had devised of his own heart; and ordained a feast unto the children of Israel: and he offered upon the altar, and burnt incense."]
]


FirstKings13 = [
    ['1Kings','1Ki','13','001','1',"And, behold, there came a man of God out of Judah by the word of the LORD unto Beth-el: and Jeroboam stood by the altar to burn incense."],
    ['1Kings','1Ki','13','002','2',"And he cried against the altar in the word of the LORD, and said, O altar, altar, thus saith the LORD; Behold, a child shall be born unto the house of David, Josiah by name; and upon thee shall he offer the priests of the high places that burn incense upon thee, and men's bones shall be burnt upon thee."],
    ['1Kings','1Ki','13','003','3',"And he gave a sign the same day, saying, This \emph{is} the sign which the LORD hath spoken; Behold, the altar shall be rent, and the ashes that \emph{are} upon it shall be poured out."],
    ['1Kings','1Ki','13','004','4',"And it came to pass, when king Jeroboam heard the saying of the man of God, which had cried against the altar in Beth-el, that he put forth his hand from the altar, saying, Lay hold on him. And his hand, which he put forth against him, dried up, so that he could not pull it in again to him."],
    ['1Kings','1Ki','13','005','5',"The altar also was rent, and the ashes poured out from the altar, according to the sign which the man of God had given by the word of the LORD."],
    ['1Kings','1Ki','13','006','6',"And the king answered and said unto the man of God, Intreat now the face of the LORD thy God, and pray for me, that my hand may be restored me again. And the man of God besought the LORD, and the king's hand was restored him again, and became as \emph{it} \emph{was} before."],
    ['1Kings','1Ki','13','007','7',"And the king said unto the man of God, Come home with me, and refresh thyself, and I will give thee a reward."],
    ['1Kings','1Ki','13','008','8',"And the man of God said unto the king, If thou wilt give me half thine house, I will not go in with thee, neither will I eat bread nor drink water in this place:"],
    ['1Kings','1Ki','13','009','9',"For so was it charged me by the word of the LORD, saying, Eat no bread, nor drink water, nor turn again by the same way that thou camest."],
    ['1Kings','1Ki','13','010','10',"So he went another way, and returned not by the way that he came to Beth-el."],
    ['1Kings','1Ki','13','011','11',"Now there dwelt an old prophet in Beth-el; and his sons came and told him all the works that the man of God had done that day in Beth-el: the words which he had spoken unto the king, them they told also to their father."],
    ['1Kings','1Ki','13','012','12',"And their father said unto them, What way went he? For his sons had seen what way the man of God went, which came from Judah."],
    ['1Kings','1Ki','13','013','13',"And he said unto his sons, Saddle me the ass. So they saddled him the ass: and he rode thereon,"],
    ['1Kings','1Ki','13','014','14',"And went after the man of God, and found him sitting under an oak: and he said unto him, \emph{Art} thou the man of God that camest from Judah? And he said, I \emph{am}."],
    ['1Kings','1Ki','13','015','15',"Then he said unto him, Come home with me, and eat bread."],
    ['1Kings','1Ki','13','016','16',"And he said, I may not return with thee, nor go in with thee: neither will I eat bread nor drink water with thee in this place:"],
    ['1Kings','1Ki','13','017','17',"For it was said to me by the word of the LORD, Thou shalt eat no bread nor drink water there, nor turn again to go by the way that thou camest."],
    ['1Kings','1Ki','13','018','18',"He said unto him, I \emph{am} a prophet also as thou \emph{art}; and an angel spake unto me by the word of the LORD, saying, Bring him back with thee into thine house, that he may eat bread and drink water. \emph{But} he lied unto him."],
    ['1Kings','1Ki','13','019','19',"So he went back with him, and did eat bread in his house, and drank water."],
    ['1Kings','1Ki','13','020','20',"And it came to pass, as they sat at the table, that the word of the LORD came unto the prophet that brought him back:"],
    ['1Kings','1Ki','13','021','21',"And he cried unto the man of God that came from Judah, saying, Thus saith the LORD, Forasmuch as thou hast disobeyed the mouth of the LORD, and hast not kept the commandment which the LORD thy God commanded thee,"],
    ['1Kings','1Ki','13','022','22',"But camest back, and hast eaten bread and drunk water in the place, of the which \emph{the} \emph{LORD} did say to thee, Eat no bread, and drink no water; thy carcase shall not come unto the sepulchre of thy fathers."],
    ['1Kings','1Ki','13','023','23',"And it came to pass, after he had eaten bread, and after he had drunk, that he saddled for him the ass, \emph{to} \emph{wit}, for the prophet whom he had brought back."],
    ['1Kings','1Ki','13','024','24',"And when he was gone, a lion met him by the way, and slew him: and his carcase was cast in the way, and the ass stood by it, the lion also stood by the carcase."],
    ['1Kings','1Ki','13','025','25',"And, behold, men passed by, and saw the carcase cast in the way, and the lion standing by the carcase: and they came and told \emph{it} in the city where the old prophet dwelt."],
    ['1Kings','1Ki','13','026','26',"And when the prophet that brought him back from the way heard \emph{thereof}, he said, It \emph{is} the man of God, who was disobedient unto the word of the LORD: therefore the LORD hath delivered him unto the lion, which hath torn him, and slain him, according to the word of the LORD, which he spake unto him."],
    ['1Kings','1Ki','13','027','27',"And he spake to his sons, saying, Saddle me the ass. And they saddled \emph{him}."],
    ['1Kings','1Ki','13','028','28',"And he went and found his carcase cast in the way, and the ass and the lion standing by the carcase: the lion had not eaten the carcase, nor torn the ass."],
    ['1Kings','1Ki','13','029','29',"And the prophet took up the carcase of the man of God, and laid it upon the ass, and brought it back: and the old prophet came to the city, to mourn and to bury him."],
    ['1Kings','1Ki','13','030','30',"And he laid his carcase in his own grave; and they mourned over him, \emph{saying}, Alas, my brother!"],
    ['1Kings','1Ki','13','031','31',"And it came to pass, after he had buried him, that he spake to his sons, saying, When I am dead, then bury me in the sepulchre wherein the man of God \emph{is} buried; lay my bones beside his bones:"],
    ['1Kings','1Ki','13','032','32',"For the saying which he cried by the word of the LORD against the altar in Beth-el, and against all the houses of the high places which \emph{are} in the cities of Samaria, shall surely come to pass."],
    ['1Kings','1Ki','13','033','33',"After this thing Jeroboam returned not from his evil way, but made again of the lowest of the people priests of the high places: whosoever would, he consecrated him, and he became \emph{one} of the priests of the high places."],
    ['1Kings','1Ki','13','034','34',"And this thing became sin unto the house of Jeroboam, even to cut \emph{it} off, and to destroy \emph{it} from off the face of the earth."]
]

FirstKings14 = [
    ['1Kings','1Ki','14','001','1',"At that time Abijah the son of Jeroboam fell sick."],
    ['1Kings','1Ki','14','002','2',"And Jeroboam said to his wife, Arise, I pray thee, and disguise thyself, that thou be not known to be the wife of Jeroboam; and get thee to Shiloh: behold, there \emph{is} Ahijah the prophet, which told me that \emph{I} \emph{should} \emph{be} king over this people."],
['1Kings','1Ki','14','003','3',"And take with thee ten loaves, and cracknels, and a cruse of honey, and go to him: he shall tell thee what shall become of the child."],
['1Kings','1Ki','14','004','4',"And Jeroboam's wife did so, and arose, and went to Shiloh, and came to the house of Ahijah. But Ahijah could not see; for his eyes were set by reason of his age."],
['1Kings','1Ki','14','005','5',"And the LORD said unto Ahijah, Behold, the wife of Jeroboam cometh to ask a thing of thee for her son; for he \emph{is} sick: thus and thus shalt thou say unto her: for it shall be, when she cometh in, that she shall feign herself \emph{to} \emph{be} another \emph{woman}."],
['1Kings','1Ki','14','006','6',"And it was \emph{so}, when Ahijah heard the sound of her feet, as she came in at the door, that he said, Come in, thou wife of Jeroboam; why feignest thou thyself \emph{to} \emph{be} another? for I \emph{am} sent to thee \emph{with} heavy \emph{tidings}."],
['1Kings','1Ki','14','007','7',"Go, tell Jeroboam, Thus saith the LORD God of Israel, Forasmuch as I exalted thee from among the people, and made thee prince over my people Israel,"],
['1Kings','1Ki','14','008','8',"And rent the kingdom away from the house of David, and gave it thee: and \emph{yet} thou hast not been as my servant David, who kept my commandments, and who followed me with all his heart, to do \emph{that} only \emph{which} \emph{was} right in mine eyes;"],
['1Kings','1Ki','14','009','9',"But hast done evil above all that were before thee: for thou hast gone and made thee other gods, and molten images, to provoke me to anger, and hast cast me behind thy back:"],
['1Kings','1Ki','14','010','10',"Therefore, behold, I will bring evil upon the house of Jeroboam, and will cut off from Jeroboam him that pisseth against the wall, \emph{and} him that is shut up and left in Israel, and will take away the remnant of the house of Jeroboam, as a man taketh away dung, till it be all gone."],
['1Kings','1Ki','14','011','11',"Him that dieth of Jeroboam in the city shall the dogs eat; and him that dieth in the field shall the fowls of the air eat: for the LORD hath spoken \emph{it}."],
['1Kings','1Ki','14','012','12',"Arise thou therefore, get thee to thine own house: \emph{and} when thy feet enter into the city, the child shall die."],
['1Kings','1Ki','14','013','13',"And all Israel shall mourn for him, and bury him: for he only of Jeroboam shall come to the grave, because in him there is found \emph{some} good thing toward the LORD God of Israel in the house of Jeroboam."],
['1Kings','1Ki','14','014','14',"Moreover the LORD shall raise him up a king over Israel, who shall cut off the house of Jeroboam that day: but what? even now."],
['1Kings','1Ki','14','015','15',"For the LORD shall smite Israel, as a reed is shaken in the water, and he shall root up Israel out of this good land, which he gave to their fathers, and shall scatter them beyond the river, because they have made their groves, provoking the LORD to anger."],
['1Kings','1Ki','14','016','16',"And he shall give Israel up because of the sins of Jeroboam, who did sin, and who made Israel to sin."],
['1Kings','1Ki','14','017','17',"And Jeroboam's wife arose, and departed, and came to Tirzah: \emph{and} when she came to the threshold of the door, the child died;"],
['1Kings','1Ki','14','018','18',"And they buried him; and all Israel mourned for him, according to the word of the LORD, which he spake by the hand of his servant Ahijah the prophet."],
['1Kings','1Ki','14','019','19',"And the rest of the acts of Jeroboam, how he warred, and how he reigned, behold, they \emph{are} written in the book of the chronicles of the kings of Israel."],
['1Kings','1Ki','14','020','20',"And the days which Jeroboam reigned \emph{were} two and twenty years: and he slept with his fathers, and Nadab his son reigned in his stead."],
['1Kings','1Ki','14','021','21',"And Rehoboam the son of Solomon reigned in Judah. Rehoboam \emph{was} forty and one years old when he began to reign, and he reigned seventeen years in Jerusalem, the city which the LORD did choose out of all the tribes of Israel, to put his name there. And his mother's name \emph{was} Naamah an Ammonitess."],
['1Kings','1Ki','14','022','22',"And Judah did evil in the sight of the LORD, and they provoked him to jealousy with their sins which they had committed, above all that their fathers had done."],
['1Kings','1Ki','14','023','23',"For they also built them high places, and images, and groves, on every high hill, and under every green tree."],
['1Kings','1Ki','14','024','24',"And there were also sodomites in the land: \emph{and} they did according to all the abominations of the nations which the LORD cast out before the children of Israel."],
['1Kings','1Ki','14','025','25',"And it came to pass in the fifth year of king Rehoboam, \emph{that} Shishak king of Egypt came up against Jerusalem:"],
['1Kings','1Ki','14','026','26',"And he took away the treasures of the house of the LORD, and the treasures of the king's house; he even took away all: and he took away all the shields of gold which Solomon had made."],
['1Kings','1Ki','14','027','27',"And king Rehoboam made in their stead brasen shields, and committed \emph{them} unto the hands of the chief of the guard, which kept the door of the king's house."],
['1Kings','1Ki','14','028','28',"And it was \emph{so}, when the king went into the house of the LORD, that the guard bare them, and brought them back into the guard chamber."],
    ['1Kings','1Ki','14','029','29',"Now the rest of the acts of Rehoboam, and all that he did, \emph{are} they not written in the book of the chronicles of the kings of Judah?"],
    ['1Kings','1Ki','14','030','30',"And there was war between Rehoboam and Jeroboam all \emph{their} days."],
    ['1Kings','1Ki','14','031','31',"And Rehoboam slept with his fathers, and was buried with his fathers in the city of David. And his mother's name \emph{was} Naamah an Ammonitess. And Abijam his son reigned in his stead."]
]

FirstKings15 = [
    ['1Kings','1Ki','15','001','1',"Now in the eighteenth year of king Jeroboam the son of Nebat reigned Abijam over Judah."],
['1Kings','1Ki','15','002','2',"Three years reigned he in Jerusalem. And his mother's name \emph{was} Maachah, the daughter of Abishalom."],
['1Kings','1Ki','15','003','3',"And he walked in all the sins of his father, which he had done before him: and his heart was not perfect with the LORD his God, as the heart of David his father."],
['1Kings','1Ki','15','004','4',"Nevertheless for David's sake did the LORD his God give him a lamp in Jerusalem, to set up his son after him, and to establish Jerusalem:"],
['1Kings','1Ki','15','005','5',"Because David did \emph{that} \emph{which} \emph{was} right in the eyes of the LORD, and turned not aside from any \emph{thing} that he commanded him all the days of his life, save only in the matter of Uriah the Hittite."],
['1Kings','1Ki','15','006','6',"And there was war between Rehoboam and Jeroboam all the days of his life."],
['1Kings','1Ki','15','007','7',"Now the rest of the acts of Abijam, and all that he did, \emph{are} they not written in the book of the chronicles of the kings of Judah? And there was war between Abijam and Jeroboam."],
['1Kings','1Ki','15','008','8',"And Abijam slept with his fathers; and they buried him in the city of David: and Asa his son reigned in his stead."],
['1Kings','1Ki','15','009','9',"And in the twentieth year of Jeroboam king of Israel reigned Asa over Judah."],
['1Kings','1Ki','15','010','10',"And forty and one years reigned he in Jerusalem. And his mother's name \emph{was} Maachah, the daughter of Abishalom."],
['1Kings','1Ki','15','011','11',"And Asa did \emph{that} \emph{which} \emph{was} right in the eyes of the LORD, as \emph{did} David his father."],
['1Kings','1Ki','15','012','12',"And he took away the sodomites out of the land, and removed all the idols that his fathers had made."],
['1Kings','1Ki','15','013','13',"And also Maachah his mother, even her he removed from \emph{being} queen, because she had made an idol in a grove; and Asa destroyed her idol, and burnt \emph{it} by the brook Kidron."],
['1Kings','1Ki','15','014','14',"But the high places were not removed: nevertheless Asa's heart was perfect with the LORD all his days."],
['1Kings','1Ki','15','015','15',"And he brought in the things which his father had dedicated, and the things which himself had dedicated, into the house of the LORD, silver, and gold, and vessels."],
['1Kings','1Ki','15','016','16',"And there was war between Asa and Baasha king of Israel all their days."],
['1Kings','1Ki','15','017','17',"And Baasha king of Israel went up against Judah, and built Ramah, that he might not suffer any to go out or come in to Asa king of Judah."],
['1Kings','1Ki','15','018','18',"Then Asa took all the silver and the gold \emph{that} \emph{were} left in the treasures of the house of the LORD, and the treasures of the king's house, and delivered them into the hand of his servants: and king Asa sent them to Ben-hadad, the son of Tabrimon, the son of Hezion, king of Syria, that dwelt at Damascus, saying,"],
['1Kings','1Ki','15','019','19',"\emph{There} \emph{is} a league between me and thee, \emph{and} between my father and thy father: behold, I have sent unto thee a present of silver and gold; come and break thy league with Baasha king of Israel, that he may depart from me."],
['1Kings','1Ki','15','020','20',"So Ben-hadad hearkened unto king Asa, and sent the captains of the hosts which he had against the cities of Israel, and smote Ijon, and Dan, and Abel-beth-maachah, and all Cinneroth, with all the land of Naphtali."],
['1Kings','1Ki','15','021','21',"And it came to pass, when Baasha heard \emph{thereof}, that he left off building of Ramah, and dwelt in Tirzah."],
['1Kings','1Ki','15','022','22',"Then king Asa made a proclamation throughout all Judah; none \emph{was} exempted: and they took away the stones of Ramah, and the timber thereof, wherewith Baasha had builded; and king Asa built with them Geba of Benjamin, and Mizpah."],
['1Kings','1Ki','15','023','23',"The rest of all the acts of Asa, and all his might, and all that he did, and the cities which he built, \emph{are} they not written in the book of the chronicles of the kings of Judah? Nevertheless in the time of his old age he was diseased in his feet."],
['1Kings','1Ki','15','024','24',"And Asa slept with his fathers, and was buried with his fathers in the city of David his father: and Jehoshaphat his son reigned in his stead."],
['1Kings','1Ki','15','025','25',"And Nadab the son of Jeroboam began to reign over Israel in the second year of Asa king of Judah, and reigned over Israel two years."],
['1Kings','1Ki','15','026','26',"And he did evil in the sight of the LORD, and walked in the way of his father, and in his sin wherewith he made Israel to sin."],
['1Kings','1Ki','15','027','27',"And Baasha the son of Ahijah, of the house of Issachar, conspired against him; and Baasha smote him at Gibbethon, which \emph{belonged} to the Philistines; for Nadab and all Israel laid siege to Gibbethon."],
['1Kings','1Ki','15','028','28',"Even in the third year of Asa king of Judah did Baasha slay him, and reigned in his stead."],
['1Kings','1Ki','15','029','29',"And it came to pass, when he reigned, \emph{that} he smote all the house of Jeroboam; he left not to Jeroboam any that breathed, until he had destroyed him, according unto the saying of the LORD, which he spake by his servant Ahijah the Shilonite:"],
['1Kings','1Ki','15','030','30',"Because of the sins of Jeroboam which he sinned, and which he made Israel sin, by his provocation wherewith he provoked the LORD God of Israel to anger."],
['1Kings','1Ki','15','031','31',"Now the rest of the acts of Nadab, and all that he did, \emph{are} they not written in the book of the chronicles of the kings of Israel?"],
    ['1Kings','1Ki','15','032','32',"And there was war between Asa and Baasha king of Israel all their days."],
    ['1Kings','1Ki','15','033','33',"In the third year of Asa king of Judah began Baasha the son of Ahijah to reign over all Israel in Tirzah, twenty and four years."],
    ['1Kings','1Ki','15','034','34',"And he did evil in the sight of the LORD, and walked in the way of Jeroboam, and in his sin wherewith he made Israel to sin."]        
]

FirstKings16 = [
    ['1Kings','1Ki','16','001','1',"Then the word of the LORD came to Jehu the son of Hanani against Baasha, saying,"],
    ['1Kings','1Ki','16','002','2',"Forasmuch as I exalted thee out of the dust, and made thee prince over my people Israel; and thou hast walked in the way of Jeroboam, and hast made my people Israel to sin, to provoke me to anger with their sins;"],
    ['1Kings','1Ki','16','003','3',"Behold, I will take away the posterity of Baasha, and the posterity of his house; and will make thy house like the house of Jeroboam the son of Nebat."],
    ['1Kings','1Ki','16','004','4',"Him that dieth of Baasha in the city shall the dogs eat; and him that dieth of his in the fields shall the fowls of the air eat."],
['1Kings','1Ki','16','005','5',"Now the rest of the acts of Baasha, and what he did, and his might, \emph{are} they not written in the book of the chronicles of the kings of Israel?"],
['1Kings','1Ki','16','006','6',"So Baasha slept with his fathers, and was buried in Tirzah: and Elah his son reigned in his stead."],
['1Kings','1Ki','16','007','7',"And also by the hand of the prophet Jehu the son of Hanani came the word of the LORD against Baasha, and against his house, even for all the evil that he did in the sight of the LORD, in provoking him to anger with the work of his hands, in being like the house of Jeroboam; and because he killed him."],
['1Kings','1Ki','16','008','8',"In the twenty and sixth year of Asa king of Judah began Elah the son of Baasha to reign over Israel in Tirzah, two years."],
['1Kings','1Ki','16','009','9',"And his servant Zimri, captain of half \emph{his} chariots, conspired against him, as he was in Tirzah, drinking himself drunk in the house of Arza steward of \emph{his} house in Tirzah."],
['1Kings','1Ki','16','010','10',"And Zimri went in and smote him, and killed him, in the twenty and seventh year of Asa king of Judah, and reigned in his stead."],
['1Kings','1Ki','16','011','11',"And it came to pass, when he began to reign, as soon as he sat on his throne, \emph{that} he slew all the house of Baasha: he left him not one that pisseth against a wall, neither of his kinsfolks, nor of his friends."],
['1Kings','1Ki','16','012','12',"Thus did Zimri destroy all the house of Baasha, according to the word of the LORD, which he spake against Baasha by Jehu the prophet,"],
['1Kings','1Ki','16','013','13',"For all the sins of Baasha, and the sins of Elah his son, by which they sinned, and by which they made Israel to sin, in provoking the LORD God of Israel to anger with their vanities."],
['1Kings','1Ki','16','014','14',"Now the rest of the acts of Elah, and all that he did, \emph{are} they not written in the book of the chronicles of the kings of Israel?"],
['1Kings','1Ki','16','015','15',"In the twenty and seventh year of Asa king of Judah did Zimri reign seven days in Tirzah. And the people \emph{were} encamped against Gibbethon, which \emph{belonged} to the Philistines."],
['1Kings','1Ki','16','016','16',"And the people \emph{that} \emph{were} encamped heard say, Zimri hath conspired, and hath also slain the king: wherefore all Israel made Omri, the captain of the host, king over Israel that day in the camp."],
['1Kings','1Ki','16','017','17',"And Omri went up from Gibbethon, and all Israel with him, and they besieged Tirzah."],
['1Kings','1Ki','16','018','18',"And it came to pass, when Zimri saw that the city was taken, that he went into the palace of the king's house, and burnt the king's house over him with fire, and died,"],
['1Kings','1Ki','16','019','19',"For his sins which he sinned in doing evil in the sight of the LORD, in walking in the way of Jeroboam, and in his sin which he did, to make Israel to sin."],
['1Kings','1Ki','16','020','20',"Now the rest of the acts of Zimri, and his treason that he wrought, \emph{are} they not written in the book of the chronicles of the kings of Israel?"],
['1Kings','1Ki','16','021','21',"Then were the people of Israel divided into two parts: half of the people followed Tibni the son of Ginath, to make him king; and half followed Omri."],
['1Kings','1Ki','16','022','22',"But the people that followed Omri prevailed against the people that followed Tibni the son of Ginath: so Tibni died, and Omri reigned."],
['1Kings','1Ki','16','023','23',"In the thirty and first year of Asa king of Judah began Omri to reign over Israel, twelve years: six years reigned he in Tirzah."],
['1Kings','1Ki','16','024','24',"And he bought the hill Samaria of Shemer for two talents of silver, and built on the hill, and called the name of the city which he built, after the name of Shemer, owner of the hill, Samaria."],
['1Kings','1Ki','16','025','25',"But Omri wrought evil in the eyes of the LORD, and did worse than all that \emph{were} before him."],
['1Kings','1Ki','16','026','26',"For he walked in all the way of Jeroboam the son of Nebat, and in his sin wherewith he made Israel to sin, to provoke the LORD God of Israel to anger with their vanities."],
['1Kings','1Ki','16','027','27',"Now the rest of the acts of Omri which he did, and his might that he shewed, \emph{are} they not written in the book of the chronicles of the kings of Israel?"],
['1Kings','1Ki','16','028','28',"So Omri slept with his fathers, and was buried in Samaria: and Ahab his son reigned in his stead."],
['1Kings','1Ki','16','029','29',"And in the thirty and eighth year of Asa king of Judah began Ahab the son of Omri to reign over Israel: and Ahab the son of Omri reigned over Israel in Samaria twenty and two years."],
['1Kings','1Ki','16','030','30',"And Ahab the son of Omri did evil in the sight of the LORD above all that \emph{were} before him."],
['1Kings','1Ki','16','031','31',"And it came to pass, as if it had been a light thing for him to walk in the sins of Jeroboam the son of Nebat, that he took to wife Jezebel the daughter of Ethbaal king of the Zidonians, and went and served Baal, and worshipped him."],
['1Kings','1Ki','16','032','32',"And he reared up an altar for Baal in the house of Baal, which he had built in Samaria."],
['1Kings','1Ki','16','033','33',"And Ahab made a grove; and Ahab did more to provoke the LORD God of Israel to anger than all the kings of Israel that were before him."],
['1Kings','1Ki','16','034','34',"In his days did Hiel the Beth-elite build Jericho: he laid the foundation thereof in Abiram his firstborn, and set up the gates thereof in his youngest \emph{son} Segub, according to the word of the LORD, which he spake by Joshua the son of Nun."]
]

FirstKings17 = [
    ['1Kings','1Ki','17','001','1',"And Elijah the Tishbite, \emph{who} \emph{was} of the inhabitants of Gilead, said unto Ahab, \emph{As} the LORD God of Israel liveth, before whom I stand, there shall not be dew nor rain these years, but according to my word."],
['1Kings','1Ki','17','002','2',"And the word of the LORD came unto him, saying,"],
['1Kings','1Ki','17','003','3',"Get thee hence, and turn thee eastward, and hide thyself by the brook Cherith, that \emph{is} before Jordan."],
['1Kings','1Ki','17','004','4',"And it shall be, \emph{that} thou shalt drink of the brook; and I have commanded the ravens to feed thee there."],
['1Kings','1Ki','17','005','5',"So he went and did according unto the word of the LORD: for he went and dwelt by the brook Cherith, that \emph{is} before Jordan."],
['1Kings','1Ki','17','006','6',"And the ravens brought him bread and flesh in the morning, and bread and flesh in the evening; and he drank of the brook."],
['1Kings','1Ki','17','007','7',"And it came to pass after a while, that the brook dried up, because there had been no rain in the land."],
['1Kings','1Ki','17','008','8',"And the word of the LORD came unto him, saying,"],
['1Kings','1Ki','17','009','9',"Arise, get thee to Zarephath, which \emph{belongeth} to Zidon, and dwell there: behold, I have commanded a widow woman there to sustain thee."],
['1Kings','1Ki','17','010','10',"So he arose and went to Zarephath. And when he came to the gate of the city, behold, the widow woman \emph{was} there gathering of sticks: and he called to her, and said, Fetch me, I pray thee, a little water in a vessel, that I may drink."],
['1Kings','1Ki','17','011','11',"And as she was going to fetch \emph{it}, he called to her, and said, Bring me, I pray thee, a morsel of bread in thine hand."],
['1Kings','1Ki','17','012','12',"And she said, \emph{As} the LORD thy God liveth, I have not a cake, but an handful of meal in a barrel, and a little oil in a cruse: and, behold, I \emph{am} gathering two sticks, that I may go in and dress it for me and my son, that we may eat it, and die."],
['1Kings','1Ki','17','013','13',"And Elijah said unto her, Fear not; go \emph{and} do as thou hast said: but make me thereof a little cake first, and bring \emph{it} unto me, and after make for thee and for thy son."],
['1Kings','1Ki','17','014','14',"For thus saith the LORD God of Israel, The barrel of meal shall not waste, neither shall the cruse of oil fail, until the day \emph{that} the LORD sendeth rain upon the earth."],
['1Kings','1Ki','17','015','15',"And she went and did according to the saying of Elijah: and she, and he, and her house, did eat \emph{many} days."],
['1Kings','1Ki','17','016','16',"\emph{And} the barrel of meal wasted not, neither did the cruse of oil fail, according to the word of the LORD, which he spake by Elijah."],
['1Kings','1Ki','17','017','17',"And it came to pass after these things, \emph{that} the son of the woman, the mistress of the house, fell sick; and his sickness was so sore, that there was no breath left in him."],
['1Kings','1Ki','17','018','18',"And she said unto Elijah, What have I to do with thee, O thou man of God? art thou come unto me to call my sin to remembrance, and to slay my son?"],
['1Kings','1Ki','17','019','19',"And he said unto her, Give me thy son. And he took him out of her bosom, and carried him up into a loft, where he abode, and laid him upon his own bed."],
['1Kings','1Ki','17','020','20',"And he cried unto the LORD, and said, O LORD my God, hast thou also brought evil upon the widow with whom I sojourn, by slaying her son?"],
['1Kings','1Ki','17','021','21',"And he stretched himself upon the child three times, and cried unto the LORD, and said, O LORD my God, I pray thee, let this child's soul come into him again."],
['1Kings','1Ki','17','022','22',"And the LORD heard the voice of Elijah; and the soul of the child came into him again, and he revived."],
['1Kings','1Ki','17','023','23',"And Elijah took the child, and brought him down out of the chamber into the house, and delivered him unto his mother: and Elijah said, See, thy son liveth."],
['1Kings','1Ki','17','024','24',"And the woman said to Elijah, Now by this I know that thou \emph{art} a man of God, \emph{and} that the word of the LORD in thy mouth \emph{is} truth."]
]

FirstKings18 = [
    ['1Kings','1Ki','18','001','1',"And it came to pass \emph{after} many days, that the word of the LORD came to Elijah in the third year, saying, Go, shew thyself unto Ahab; and I will send rain upon the earth."],
['1Kings','1Ki','18','002','2',"And Elijah went to shew himself unto Ahab. And \emph{there} \emph{was} a sore famine in Samaria."],
['1Kings','1Ki','18','003','3',"And Ahab called Obadiah, which \emph{was} the governor of \emph{his} house. (Now Obadiah feared the LORD greatly:"],
['1Kings','1Ki','18','004','4',"For it was \emph{so}, when Jezebel cut off the prophets of the LORD, that Obadiah took an hundred prophets, and hid them by fifty in a cave, and fed them with bread and water.)"],
['1Kings','1Ki','18','005','5',"And Ahab said unto Obadiah, Go into the land, unto all fountains of water, and unto all brooks: peradventure we may find grass to save the horses and mules alive, that we lose not all the beasts."],
['1Kings','1Ki','18','006','6',"So they divided the land between them to pass throughout it: Ahab went one way by himself, and Obadiah went another way by himself."],
['1Kings','1Ki','18','007','7',"And as Obadiah was in the way, behold, Elijah met him: and he knew him, and fell on his face, and said, \emph{Art} thou that my lord Elijah?"],
['1Kings','1Ki','18','008','8',"And he answered him, I \emph{am}: go, tell thy lord, Behold, Elijah \emph{is} \emph{here}."],
['1Kings','1Ki','18','009','9',"And he said, What have I sinned, that thou wouldest deliver thy servant into the hand of Ahab, to slay me?"],
['1Kings','1Ki','18','010','10',"\emph{As} the LORD thy God liveth, there is no nation or kingdom, whither my lord hath not sent to seek thee: and when they said, \emph{He} \emph{is} not \emph{there}; he took an oath of the kingdom and nation, that they found thee not."],
['1Kings','1Ki','18','011','11',"And now thou sayest, Go, tell thy lord, Behold, Elijah \emph{is} \emph{here}."],
['1Kings','1Ki','18','012','12',"And it shall come to pass, \emph{as} \emph{soon} \emph{as} I am gone from thee, that the Spirit of the LORD shall carry thee whither I know not; and \emph{so} when I come and tell Ahab, and he cannot find thee, he shall slay me: but I thy servant fear the LORD from my youth."],
['1Kings','1Ki','18','013','13',"Was it not told my lord what I did when Jezebel slew the prophets of the LORD, how I hid an hundred men of the LORD'S prophets by fifty in a cave, and fed them with bread and water?"],
['1Kings','1Ki','18','014','14',"And now thou sayest, Go, tell thy lord, Behold, Elijah \emph{is} \emph{here}: and he shall slay me."],
['1Kings','1Ki','18','015','15',"And Elijah said, \emph{As} the LORD of hosts liveth, before whom I stand, I will surely shew myself unto him to day."],
['1Kings','1Ki','18','016','16',"So Obadiah went to meet Ahab, and told him: and Ahab went to meet Elijah."],
['1Kings','1Ki','18','017','17',"And it came to pass, when Ahab saw Elijah, that Ahab said unto him, \emph{Art} thou he that troubleth Israel?"],
['1Kings','1Ki','18','018','18',"And he answered, I have not troubled Israel; but thou, and thy father's house, in that ye have forsaken the commandments of the LORD, and thou hast followed Baalim."],
['1Kings','1Ki','18','019','19',"Now therefore send, \emph{and} gather to me all Israel unto mount Carmel, and the prophets of Baal four hundred and fifty, and the prophets of the groves four hundred, which eat at Jezebel's table."],
['1Kings','1Ki','18','020','20',"So Ahab sent unto all the children of Israel, and gathered the prophets together unto mount Carmel."],
['1Kings','1Ki','18','021','21',"And Elijah came unto all the people, and said, How long halt ye between two opinions? if the LORD \emph{be} God, follow him: but if Baal, \emph{then} follow him. And the people answered him not a word."],
['1Kings','1Ki','18','022','22',"Then said Elijah unto the people, I, \emph{even} I only, remain a prophet of the LORD; but Baal's prophets \emph{are} four hundred and fifty men."],
['1Kings','1Ki','18','023','23',"Let them therefore give us two bullocks; and let them choose one bullock for themselves, and cut it in pieces, and lay \emph{it} on wood, and put no fire \emph{under}: and I will dress the other bullock, and lay \emph{it} on wood, and put no fire \emph{under}:"],
['1Kings','1Ki','18','024','24',"And call ye on the name of your gods, and I will call on the name of the LORD: and the God that answereth by fire, let him be God. And all the people answered and said, It is well spoken."],
['1Kings','1Ki','18','025','25',"And Elijah said unto the prophets of Baal, Choose you one bullock for yourselves, and dress \emph{it} first; for ye \emph{are} many; and call on the name of your gods, but put no fire \emph{under}."],
['1Kings','1Ki','18','026','26',"And they took the bullock which was given them, and they dressed \emph{it}, and called on the name of Baal from morning even until noon, saying, O Baal, hear us. But \emph{there} \emph{was} no voice, nor any that answered. And they leaped upon the altar which was made."],
['1Kings','1Ki','18','027','27',"And it came to pass at noon, that Elijah mocked them, and said, Cry aloud: for he \emph{is} a god; either he is talking, or he is pursuing, or he is in a journey, \emph{or} peradventure he sleepeth, and must be awaked."],
['1Kings','1Ki','18','028','28',"And they cried aloud, and cut themselves after their manner with knives and lancets, till the blood gushed out upon them."],
['1Kings','1Ki','18','029','29',"And it came to pass, when midday was past, and they prophesied until the \emph{time} of the offering of the \emph{evening} sacrifice, that \emph{there} \emph{was} neither voice, nor any to answer, nor any that regarded."],
['1Kings','1Ki','18','030','30',"And Elijah said unto all the people, Come near unto me. And all the people came near unto him. And he repaired the altar of the LORD \emph{that} \emph{was} broken down."],
['1Kings','1Ki','18','031','31',"And Elijah took twelve stones, according to the number of the tribes of the sons of Jacob, unto whom the word of the LORD came, saying, Israel shall be thy name:"],
['1Kings','1Ki','18','032','32',"And with the stones he built an altar in the name of the LORD: and he made a trench about the altar, as great as would contain two measures of seed."],
['1Kings','1Ki','18','033','33',"And he put the wood in order, and cut the bullock in pieces, and laid \emph{him} on the wood, and said, Fill four barrels with water, and pour \emph{it} on the burnt sacrifice, and on the wood."],
['1Kings','1Ki','18','034','34',"And he said, Do \emph{it} the second time. And they did \emph{it} the second time. And he said, Do \emph{it} the third time. And they did \emph{it} the third time."],
['1Kings','1Ki','18','035','35',"And the water ran round about the altar; and he filled the trench also with water."],
['1Kings','1Ki','18','036','36',"And it came to pass at \emph{the} \emph{time} \emph{of} the offering of the \emph{evening} sacrifice, that Elijah the prophet came near, and said, LORD God of Abraham, Isaac, and of Israel, let it be known this day that thou \emph{art} God in Israel, and \emph{that} I \emph{am} thy servant, and \emph{that} I have done all these things at thy word."],
['1Kings','1Ki','18','037','37',"Hear me, O LORD, hear me, that this people may know that thou \emph{art} the LORD God, and \emph{that} thou hast turned their heart back again."],
['1Kings','1Ki','18','038','38',"Then the fire of the LORD fell, and consumed the burnt sacrifice, and the wood, and the stones, and the dust, and licked up the water that \emph{was} in the trench."],
['1Kings','1Ki','18','039','39',"And when all the people saw \emph{it}, they fell on their faces: and they said, The LORD, he \emph{is} the God; the LORD, he \emph{is} the God."],
['1Kings','1Ki','18','040','40',"And Elijah said unto them, Take the prophets of Baal; let not one of them escape. And they took them: and Elijah brought them down to the brook Kishon, and slew them there."],
['1Kings','1Ki','18','041','41',"And Elijah said unto Ahab, Get thee up, eat and drink; for \emph{there} \emph{is} a sound of abundance of rain."],
['1Kings','1Ki','18','042','42',"So Ahab went up to eat and to drink. And Elijah went up to the top of Carmel; and he cast himself down upon the earth, and put his face between his knees,"],
['1Kings','1Ki','18','043','43',"And said to his servant, Go up now, look toward the sea. And he went up, and looked, and said, \emph{There} \emph{is} nothing. And he said, Go again seven times."],
['1Kings','1Ki','18','044','44',"And it came to pass at the seventh time, that he said, Behold, there ariseth a little cloud out of the sea, like a man's hand. And he said, Go up, say unto Ahab, Prepare \emph{thy} \emph{chariot}, and get thee down, that the rain stop thee not."],
['1Kings','1Ki','18','045','45',"And it came to pass in the mean while, that the heaven was black with clouds and wind, and there was a great rain. And Ahab rode, and went to Jezreel."],
['1Kings','1Ki','18','046','46',"And the hand of the LORD was on Elijah; and he girded up his loins, and ran before Ahab to the entrance of Jezreel."]
]

FirstKings19 = [
    ['1Kings','1Ki','19','001','1',"And Ahab told Jezebel all that Elijah had done, and withal how he had slain all the prophets with the sword."],
['1Kings','1Ki','19','002','2',"Then Jezebel sent a messenger unto Elijah, saying, So let the gods do \emph{to} \emph{me}, and more also, if I make not thy life as the life of one of them by to morrow about this time."],
['1Kings','1Ki','19','003','3',"And when he saw \emph{that}, he arose, and went for his life, and came to Beer-sheba, which \emph{belongeth} to Judah, and left his servant there."],
['1Kings','1Ki','19','004','4',"But he himself went a day's journey into the wilderness, and came and sat down under a juniper tree: and he requested for himself that he might die; and said, It is enough; now, O LORD, take away my life; for I \emph{am} not better than my fathers."],
['1Kings','1Ki','19','005','5',"And as he lay and slept under a juniper tree, behold, then an angel touched him, and said unto him, Arise \emph{and} eat."],
['1Kings','1Ki','19','006','6',"And he looked, and, behold, \emph{there} \emph{was} a cake baken on the coals, and a cruse of water at his head. And he did eat and drink, and laid him down again."],
['1Kings','1Ki','19','007','7',"And the angel of the LORD came again the second time, and touched him, and said, Arise \emph{and} eat; because the journey \emph{is} too great for thee."],
['1Kings','1Ki','19','008','8',"And he arose, and did eat and drink, and went in the strength of that meat forty days and forty nights unto Horeb the mount of God."],
['1Kings','1Ki','19','009','9',"And he came thither unto a cave, and lodged there; and, behold, the word of the LORD \emph{came} to him, and he said unto him, What doest thou here, Elijah?"],
['1Kings','1Ki','19','010','10',"And he said, I have been very jealous for the LORD God of hosts: for the children of Israel have forsaken thy covenant, thrown down thine altars, and slain thy prophets with the sword; and I, \emph{even} I only, am left; and they seek my life, to take it away."],
['1Kings','1Ki','19','011','11',"And he said, Go forth, and stand upon the mount before the LORD. And, behold, the LORD passed by, and a great and strong wind rent the mountains, and brake in pieces the rocks before the LORD; \emph{but} the LORD \emph{was} not in the wind: and after the wind an earthquake; \emph{but} the LORD \emph{was} not in the earthquake:"],
['1Kings','1Ki','19','012','12',"And after the earthquake a fire; \emph{but} the LORD \emph{was} not in the fire: and after the fire a still small voice."],
['1Kings','1Ki','19','013','13',"And it was \emph{so}, when Elijah heard \emph{it}, that he wrapped his face in his mantle, and went out, and stood in the entering in of the cave. And, behold, \emph{there} \emph{came} a voice unto him, and said, What doest thou here, Elijah?"],
['1Kings','1Ki','19','014','14',"And he said, I have been very jealous for the LORD God of hosts: because the children of Israel have forsaken thy covenant, thrown down thine altars, and slain thy prophets with the sword; and I, \emph{even} I only, am left; and they seek my life, to take it away."],
['1Kings','1Ki','19','015','15',"And the LORD said unto him, Go, return on thy way to the wilderness of Damascus: and when thou comest, anoint Hazael \emph{to} \emph{be} king over Syria:"],
['1Kings','1Ki','19','016','16',"And Jehu the son of Nimshi shalt thou anoint \emph{to} \emph{be} king over Israel: and Elisha the son of Shaphat of Abelmeholah shalt thou anoint \emph{to} \emph{be} prophet in thy room."],
['1Kings','1Ki','19','017','17',"And it shall come to pass, \emph{that} him that escapeth the sword of Hazael shall Jehu slay: and him that escapeth from the sword of Jehu shall Elisha slay."],
['1Kings','1Ki','19','018','18',"Yet I have left \emph{me} seven thousand in Israel, all the knees which have not bowed unto Baal, and every mouth which hath not kissed him."],
['1Kings','1Ki','19','019','19',"So he departed thence, and found Elisha the son of Shaphat, who \emph{was} plowing \emph{with} twelve yoke \emph{of} \emph{oxen} before him, and he with the twelfth: and Elijah passed by him, and cast his mantle upon him."],
['1Kings','1Ki','19','020','20',"And he left the oxen, and ran after Elijah, and said, Let me, I pray thee, kiss my father and my mother, and \emph{then} I will follow thee. And he said unto him, Go back again: for what have I done to thee?"],
['1Kings','1Ki','19','021','21',"And he returned back from him, and took a yoke of oxen, and slew them, and boiled their flesh with the instruments of the oxen, and gave unto the people, and they did eat. Then he arose, and went after Elijah, and ministered unto him."]
]

FirstKings20 = [
    ['1Kings','1Ki','20','001','1',"And Ben-hadad the king of Syria gathered all his host together: and \emph{there} \emph{were} thirty and two kings with him, and horses, and chariots: and he went up and besieged Samaria, and warred against it."],
['1Kings','1Ki','20','002','2',"And he sent messengers to Ahab king of Israel into the city, and said unto him, Thus saith Ben-hadad,"],
['1Kings','1Ki','20','003','3',"Thy silver and thy gold \emph{is} mine; thy wives also and thy children, \emph{even} the goodliest, \emph{are} mine."],
['1Kings','1Ki','20','004','4',"And the king of Israel answered and said, My lord, O king, according to thy saying, I \emph{am} thine, and all that I have."],
['1Kings','1Ki','20','005','5',"And the messengers came again, and said, Thus speaketh Ben-hadad, saying, Although I have sent unto thee, saying, Thou shalt deliver me thy silver, and thy gold, and thy wives, and thy children;"],
['1Kings','1Ki','20','006','6',"Yet I will send my servants unto thee to morrow about this time, and they shall search thine house, and the houses of thy servants; and it shall be, \emph{that} whatsoever is pleasant in thine eyes, they shall put \emph{it} in their hand, and take \emph{it} away."],
['1Kings','1Ki','20','007','7',"Then the king of Israel called all the elders of the land, and said, Mark, I pray you, and see how this \emph{man} seeketh mischief: for he sent unto me for my wives, and for my children, and for my silver, and for my gold; and I denied him not."],
['1Kings','1Ki','20','008','8',"And all the elders and all the people said unto him, Hearken not \emph{unto} \emph{him}, nor consent."],
['1Kings','1Ki','20','009','9',"Wherefore he said unto the messengers of Ben-hadad, Tell my lord the king, All that thou didst send for to thy servant at the first I will do: but this thing I may not do. And the messengers departed, and brought him word again."],
['1Kings','1Ki','20','010','10',"And Ben-hadad sent unto him, and said, The gods do so unto me, and more also, if the dust of Samaria shall suffice for handfuls for all the people that follow me."],
['1Kings','1Ki','20','011','11',"And the king of Israel answered and said, Tell \emph{him}, Let not him that girdeth on \emph{his} \emph{harness} boast himself as he that putteth it off."],
['1Kings','1Ki','20','012','12',"And it came to pass, when \emph{Ben-hadad} heard this message, as he \emph{was} drinking, he and the kings in the pavilions, that he said unto his servants, Set \emph{yourselves} \emph{in} \emph{array}. And they set \emph{themselves} \emph{in} \emph{array} against the city."],
['1Kings','1Ki','20','013','13',"And, behold, there came a prophet unto Ahab king of Israel, saying, Thus saith the LORD, Hast thou seen all this great multitude? behold, I will deliver it into thine hand this day; and thou shalt know that I \emph{am} the LORD."],
['1Kings','1Ki','20','014','14',"And Ahab said, By whom? And he said, Thus saith the LORD, \emph{Even} by the young men of the princes of the provinces. Then he said, Who shall order the battle? And he answered, Thou."],
['1Kings','1Ki','20','015','15',"Then he numbered the young men of the princes of the provinces, and they were two hundred and thirty two: and after them he numbered all the people, \emph{even} all the children of Israel, \emph{being} seven thousand."],
['1Kings','1Ki','20','016','16',"And they went out at noon. But Ben-hadad \emph{was} drinking himself drunk in the pavilions, he and the kings, the thirty and two kings that helped him."],
['1Kings','1Ki','20','017','17',"And the young men of the princes of the provinces went out first; and Ben-hadad sent out, and they told him, saying, There are men come out of Samaria."],
['1Kings','1Ki','20','018','18',"And he said, Whether they be come out for peace, take them alive; or whether they be come out for war, take them alive."],
['1Kings','1Ki','20','019','19',"So these young men of the princes of the provinces came out of the city, and the army which followed them."],
['1Kings','1Ki','20','020','20',"And they slew every one his man: and the Syrians fled; and Israel pursued them: and Ben-hadad the king of Syria escaped on an horse with the horsemen."],
['1Kings','1Ki','20','021','21',"And the king of Israel went out, and smote the horses and chariots, and slew the Syrians with a great slaughter."],
['1Kings','1Ki','20','022','22',"And the prophet came to the king of Israel, and said unto him, Go, strengthen thyself, and mark, and see what thou doest: for at the return of the year the king of Syria will come up against thee."],
['1Kings','1Ki','20','023','23',"And the servants of the king of Syria said unto him, Their gods \emph{are} gods of the hills; therefore they were stronger than we; but let us fight against them in the plain, and surely we shall be stronger than they."],
['1Kings','1Ki','20','024','24',"And do this thing, Take the kings away, every man out of his place, and put captains in their rooms:"],
['1Kings','1Ki','20','025','25',"And number thee an army, like the army that thou hast lost, horse for horse, and chariot for chariot: and we will fight against them in the plain, \emph{and} surely we shall be stronger than they. And he hearkened unto their voice, and did so."],
['1Kings','1Ki','20','026','26',"And it came to pass at the return of the year, that Ben-hadad numbered the Syrians, and went up to Aphek, to fight against Israel."],
['1Kings','1Ki','20','027','27',"And the children of Israel were numbered, and were all present, and went against them: and the children of Israel pitched before them like two little flocks of kids; but the Syrians filled the country."],
['1Kings','1Ki','20','028','28',"And there came a man of God, and spake unto the king of Israel, and said, Thus saith the LORD, Because the Syrians have said, The LORD \emph{is} God of the hills, but he \emph{is} not God of the valleys, therefore will I deliver all this great multitude into thine hand, and ye shall know that I \emph{am} the LORD."],
['1Kings','1Ki','20','029','29',"And they pitched one over against the other seven days. And \emph{so} it was, that in the seventh day the battle was joined: and the children of Israel slew of the Syrians an hundred thousand footmen in one day."],
['1Kings','1Ki','20','030','30',"But the rest fled to Aphek, into the city; and \emph{there} a wall fell upon twenty and seven thousand of the men \emph{that} \emph{were} left. And Ben-hadad fled, and came into the city, into an inner chamber."],
['1Kings','1Ki','20','031','31',"And his servants said unto him, Behold now, we have heard that the kings of the house of Israel \emph{are} merciful kings: let us, I pray thee, put sackcloth on our loins, and ropes upon our heads, and go out to the king of Israel: peradventure he will save thy life."],
['1Kings','1Ki','20','032','32',"So they girded sackcloth on their loins, and \emph{put} ropes on their heads, and came to the king of Israel, and said, Thy servant Ben-hadad saith, I pray thee, let me live. And he said, \emph{Is} he yet alive? he \emph{is} my brother."],
['1Kings','1Ki','20','033','33',"Now the men did diligently observe whether \emph{any} \emph{thing} \emph{would} \emph{come} from him, and did hastily catch \emph{it}: and they said, Thy brother Ben-hadad. Then he said, Go ye, bring him. Then Ben-hadad came forth to him; and he caused him to come up into the chariot."],
['1Kings','1Ki','20','034','34',"And \emph{Ben-hadad} said unto him, The cities, which my father took from thy father, I will restore; and thou shalt make streets for thee in Damascus, as my father made in Samaria. Then \emph{said} \emph{Ahab}, I will send thee away with this covenant. So he made a covenant with him, and sent him away."],
['1Kings','1Ki','20','035','35',"And a certain man of the sons of the prophets said unto his neighbour in the word of the LORD, Smite me, I pray thee. And the man refused to smite him."],
['1Kings','1Ki','20','036','36',"Then said he unto him, Because thou hast not obeyed the voice of the LORD, behold, as soon as thou art departed from me, a lion shall slay thee. And as soon as he was departed from him, a lion found him, and slew him."],
['1Kings','1Ki','20','037','37',"Then he found another man, and said, Smite me, I pray thee. And the man smote him, so that in smiting he wounded \emph{him}."],
['1Kings','1Ki','20','038','38',"So the prophet departed, and waited for the king by the way, and disguised himself with ashes upon his face."],
['1Kings','1Ki','20','039','39',"And as the king passed by, he cried unto the king: and he said, Thy servant went out into the midst of the battle; and, behold, a man turned aside, and brought a man unto me, and said, Keep this man: if by any means he be missing, then shall thy life be for his life, or else thou shalt pay a talent of silver."],
['1Kings','1Ki','20','040','40',"And as thy servant was busy here and there, he was gone. And the king of Israel said unto him, So \emph{shall} thy judgment \emph{be}; thyself hast decided \emph{it}."],
['1Kings','1Ki','20','041','41',"And he hasted, and took the ashes away from his face; and the king of Israel discerned him that he \emph{was} of the prophets."],
['1Kings','1Ki','20','042','42',"And he said unto him, Thus saith the LORD, Because thou hast let go out of \emph{thy} hand a man whom I appointed to utter destruction, therefore thy life shall go for his life, and thy people for his people."],
['1Kings','1Ki','20','043','43',"And the king of Israel went to his house heavy and displeased, and came to Samaria."]
]

FirstKings21 = [
    ['1Kings','1Ki','21','001','1',"And it came to pass after these things, \emph{that} Naboth the Jezreelite had a vineyard, which \emph{was} in Jezreel, hard by the palace of Ahab king of Samaria."],
['1Kings','1Ki','21','002','2',"And Ahab spake unto Naboth, saying, Give me thy vineyard, that I may have it for a garden of herbs, because it \emph{is} near unto my house: and I will give thee for it a better vineyard than it; \emph{or}, if it seem good to thee, I will give thee the worth of it in money."],
['1Kings','1Ki','21','003','3',"And Naboth said to Ahab, The LORD forbid it me, that I should give the inheritance of my fathers unto thee."],
['1Kings','1Ki','21','004','4',"And Ahab came into his house heavy and displeased because of the word which Naboth the Jezreelite had spoken to him: for he had said, I will not give thee the inheritance of my fathers. And he laid him down upon his bed, and turned away his face, and would eat no bread."],
['1Kings','1Ki','21','005','5',"But Jezebel his wife came to him, and said unto him, Why is thy spirit so sad, that thou eatest no bread?"],
['1Kings','1Ki','21','006','6',"And he said unto her, Because I spake unto Naboth the Jezreelite, and said unto him, Give me thy vineyard for money; or else, if it please thee, I will give thee \emph{another} vineyard for it: and he answered, I will not give thee my vineyard."],
['1Kings','1Ki','21','007','7',"And Jezebel his wife said unto him, Dost thou now govern the kingdom of Israel? arise, \emph{and} eat bread, and let thine heart be merry: I will give thee the vineyard of Naboth the Jezreelite."],
['1Kings','1Ki','21','008','8',"So she wrote letters in Ahab's name, and sealed \emph{them} with his seal, and sent the letters unto the elders and to the nobles that \emph{were} in his city, dwelling with Naboth."],
['1Kings','1Ki','21','009','9',"And she wrote in the letters, saying, Proclaim a fast, and set Naboth on high among the people:"],
['1Kings','1Ki','21','010','10',"And set two men, sons of Belial, before him, to bear witness against him, saying, Thou didst blaspheme God and the king. And \emph{then} carry him out, and stone him, that he may die."],
['1Kings','1Ki','21','011','11',"And the men of his city, \emph{even} the elders and the nobles who were the inhabitants in his city, did as Jezebel had sent unto them, \emph{and} as it \emph{was} written in the letters which she had sent unto them."],
['1Kings','1Ki','21','012','12',"They proclaimed a fast, and set Naboth on high among the people."],
['1Kings','1Ki','21','013','13',"And there came in two men, children of Belial, and sat before him: and the men of Belial witnessed against him, \emph{even} against Naboth, in the presence of the people, saying, Naboth did blaspheme God and the king. Then they carried him forth out of the city, and stoned him with stones, that he died."],
['1Kings','1Ki','21','014','14',"Then they sent to Jezebel, saying, Naboth is stoned, and is dead."],
['1Kings','1Ki','21','015','15',"And it came to pass, when Jezebel heard that Naboth was stoned, and was dead, that Jezebel said to Ahab, Arise, take possession of the vineyard of Naboth the Jezreelite, which he refused to give thee for money: for Naboth is not alive, but dead."],
['1Kings','1Ki','21','016','16',"And it came to pass, when Ahab heard that Naboth was dead, that Ahab rose up to go down to the vineyard of Naboth the Jezreelite, to take possession of it."],
['1Kings','1Ki','21','017','17',"And the word of the LORD came to Elijah the Tishbite, saying,"],
['1Kings','1Ki','21','018','18',"Arise, go down to meet Ahab king of Israel, which \emph{is} in Samaria: behold, \emph{he} \emph{is} in the vineyard of Naboth, whither he is gone down to possess it."],
['1Kings','1Ki','21','019','19',"And thou shalt speak unto him, saying, Thus saith the LORD, Hast thou killed, and also taken possession? And thou shalt speak unto him, saying, Thus saith the LORD, In the place where dogs licked the blood of Naboth shall dogs lick thy blood, even thine."],
['1Kings','1Ki','21','020','20',"And Ahab said to Elijah, Hast thou found me, O mine enemy? And he answered, I have found \emph{thee}: because thou hast sold thyself to work evil in the sight of the LORD."],
['1Kings','1Ki','21','021','21',"Behold, I will bring evil upon thee, and will take away thy posterity, and will cut off from Ahab him that pisseth against the wall, and him that is shut up and left in Israel,"],
['1Kings','1Ki','21','022','22',"And will make thine house like the house of Jeroboam the son of Nebat, and like the house of Baasha the son of Ahijah, for the provocation wherewith thou hast provoked \emph{me} to anger, and made Israel to sin."],
['1Kings','1Ki','21','023','23',"And of Jezebel also spake the LORD, saying, The dogs shall eat Jezebel by the wall of Jezreel."],
['1Kings','1Ki','21','024','24',"Him that dieth of Ahab in the city the dogs shall eat; and him that dieth in the field shall the fowls of the air eat."],
['1Kings','1Ki','21','025','25',"But there was none like unto Ahab, which did sell himself to work wickedness in the sight of the LORD, whom Jezebel his wife stirred up."],
['1Kings','1Ki','21','026','26',"And he did very abominably in following idols, according to all \emph{things} as did the Amorites, whom the LORD cast out before the children of Israel."],
['1Kings','1Ki','21','027','27',"And it came to pass, when Ahab heard those words, that he rent his clothes, and put sackcloth upon his flesh, and fasted, and lay in sackcloth, and went softly."],
['1Kings','1Ki','21','028','28',"And the word of the LORD came to Elijah the Tishbite, saying,"],
['1Kings','1Ki','21','029','29',"Seest thou how Ahab humbleth himself before me? because he humbleth himself before me, I will not bring the evil in his days: \emph{but} in his son's days will I bring the evil upon his house."]
]

FirstKings22 = [
    ['1Kings','1Ki','22','001','1',"And they continued three years without war between Syria and Israel."],
['1Kings','1Ki','22','002','2',"And it came to pass in the third year, that Jehoshaphat the king of Judah came down to the king of Israel."],
['1Kings','1Ki','22','003','3',"And the king of Israel said unto his servants, Know ye that Ramoth in Gilead \emph{is} our's, and we \emph{be} still, \emph{and} take it not out of the hand of the king of Syria?"],
['1Kings','1Ki','22','004','4',"And he said unto Jehoshaphat, Wilt thou go with me to battle to Ramoth-gilead? And Jehoshaphat said to the king of Israel, I \emph{am} as thou \emph{art}, my people as thy people, my horses as thy horses."],
['1Kings','1Ki','22','005','5',"And Jehoshaphat said unto the king of Israel, Enquire, I pray thee, at the word of the LORD to day."],
['1Kings','1Ki','22','006','6',"Then the king of Israel gathered the prophets together, about four hundred men, and said unto them, Shall I go against Ramoth-gilead to battle, or shall I forbear? And they said, Go up; for the Lord shall deliver \emph{it} into the hand of the king."],
['1Kings','1Ki','22','007','7',"And Jehoshaphat said, \emph{Is} \emph{there} not here a prophet of the LORD besides, that we might enquire of him?"],
['1Kings','1Ki','22','008','8',"And the king of Israel said unto Jehoshaphat, \emph{There} \emph{is} yet one man, Micaiah the son of Imlah, by whom we may enquire of the LORD: but I hate him; for he doth not prophesy good concerning me, but evil. And Jehoshaphat said, Let not the king say so."],
['1Kings','1Ki','22','009','9',"Then the king of Israel called an officer, and said, Hasten \emph{hither} Micaiah the son of Imlah."],
['1Kings','1Ki','22','010','10',"And the king of Israel and Jehoshaphat the king of Judah sat each on his throne, having put on their robes, in a void place in the entrance of the gate of Samaria; and all the prophets prophesied before them."],
['1Kings','1Ki','22','011','11',"And Zedekiah the son of Chenaanah made him horns of iron: and he said, Thus saith the LORD, With these shalt thou push the Syrians, until thou have consumed them."],
['1Kings','1Ki','22','012','12',"And all the prophets prophesied so, saying, Go up to Ramoth-gilead, and prosper: for the LORD shall deliver \emph{it} into the king's hand."],
['1Kings','1Ki','22','013','13',"And the messenger that was gone to call Micaiah spake unto him, saying, Behold now, the words of the prophets \emph{declare} good unto the king with one mouth: let thy word, I pray thee, be like the word of one of them, and speak \emph{that} \emph{which} \emph{is} good."],
['1Kings','1Ki','22','014','14',"And Micaiah said, \emph{As} the LORD liveth, what the LORD saith unto me, that will I speak."],
['1Kings','1Ki','22','015','15',"So he came to the king. And the king said unto him, Micaiah, shall we go against Ramoth-gilead to battle, or shall we forbear? And he answered him, Go, and prosper: for the LORD shall deliver \emph{it} into the hand of the king."],
['1Kings','1Ki','22','016','16',"And the king said unto him, How many times shall I adjure thee that thou tell me nothing but \emph{that} \emph{which} \emph{is} true in the name of the LORD?"],
['1Kings','1Ki','22','017','17',"And he said, I saw all Israel scattered upon the hills, as sheep that have not a shepherd: and the LORD said, These have no master: let them return every man to his house in peace."],
['1Kings','1Ki','22','018','18',"And the king of Israel said unto Jehoshaphat, Did I not tell thee that he would prophesy no good concerning me, but evil?"],
['1Kings','1Ki','22','019','19',"And he said, Hear thou therefore the word of the LORD: I saw the LORD sitting on his throne, and all the host of heaven standing by him on his right hand and on his left."],
['1Kings','1Ki','22','020','20',"And the LORD said, Who shall persuade Ahab, that he may go up and fall at Ramoth-gilead? And one said on this manner, and another said on that manner."],
['1Kings','1Ki','22','021','21',"And there came forth a spirit, and stood before the LORD, and said, I will persuade him."],
['1Kings','1Ki','22','022','22',"And the LORD said unto him, Wherewith? And he said, I will go forth, and I will be a lying spirit in the mouth of all his prophets. And he said, Thou shalt persuade \emph{him}, and prevail also: go forth, and do so."],
['1Kings','1Ki','22','023','23',"Now therefore, behold, the LORD hath put a lying spirit in the mouth of all these thy prophets, and the LORD hath spoken evil concerning thee."],
['1Kings','1Ki','22','024','24',"But Zedekiah the son of Chenaanah went near, and smote Micaiah on the cheek, and said, Which way went the Spirit of the LORD from me to speak unto thee?"],
['1Kings','1Ki','22','025','25',"And Micaiah said, Behold, thou shalt see in that day, when thou shalt go into an inner chamber to hide thyself."],
['1Kings','1Ki','22','026','26',"And the king of Israel said, Take Micaiah, and carry him back unto Amon the governor of the city, and to Joash the king's son;"],
['1Kings','1Ki','22','027','27',"And say, Thus saith the king, Put this \emph{fellow} in the prison, and feed him with bread of affliction and with water of affliction, until I come in peace."],
['1Kings','1Ki','22','028','28',"And Micaiah said, If thou return at all in peace, the LORD hath not spoken by me. And he said, Hearken, O people, every one of you."],
['1Kings','1Ki','22','029','29',"So the king of Israel and Jehoshaphat the king of Judah went up to Ramoth-gilead."],
['1Kings','1Ki','22','030','30',"And the king of Israel said unto Jehoshaphat, I will disguise myself, and enter into the battle; but put thou on thy robes. And the king of Israel disguised himself, and went into the battle."],
['1Kings','1Ki','22','031','31',"But the king of Syria commanded his thirty and two captains that had rule over his chariots, saying, Fight neither with small nor great, save only with the king of Israel."],
['1Kings','1Ki','22','032','32',"And it came to pass, when the captains of the chariots saw Jehoshaphat, that they said, Surely it \emph{is} the king of Israel. And they turned aside to fight against him: and Jehoshaphat cried out."],
['1Kings','1Ki','22','033','33',"And it came to pass, when the captains of the chariots perceived that it \emph{was} not the king of Israel, that they turned back from pursuing him."],
['1Kings','1Ki','22','034','34',"And a \emph{certain} man drew a bow at a venture, and smote the king of Israel between the joints of the harness: wherefore he said unto the driver of his chariot, Turn thine hand, and carry me out of the host; for I am wounded."],
['1Kings','1Ki','22','035','35',"And the battle increased that day: and the king was stayed up in his chariot against the Syrians, and died at even: and the blood ran out of the wound into the midst of the chariot."],
['1Kings','1Ki','22','036','36',"And there went a proclamation throughout the host about the going down of the sun, saying, Every man to his city, and every man to his own country."],
['1Kings','1Ki','22','037','37',"So the king died, and was brought to Samaria; and they buried the king in Samaria."],
['1Kings','1Ki','22','038','38',"And \emph{one} washed the chariot in the pool of Samaria; and the dogs licked up his blood; and they washed his armour; according unto the word of the LORD which he spake."],
['1Kings','1Ki','22','039','39',"Now the rest of the acts of Ahab, and all that he did, and the ivory house which he made, and all the cities that he built, \emph{are} they not written in the book of the chronicles of the kings of Israel?"],
['1Kings','1Ki','22','040','40',"So Ahab slept with his fathers; and Ahaziah his son reigned in his stead."],
['1Kings','1Ki','22','041','41',"And Jehoshaphat the son of Asa began to reign over Judah in the fourth year of Ahab king of Israel."],
['1Kings','1Ki','22','042','42',"Jehoshaphat \emph{was} thirty and five years old when he began to reign; and he reigned twenty and five years in Jerusalem. And his mother's name \emph{was} Azubah the daughter of Shilhi."],
['1Kings','1Ki','22','043','43',"And he walked in all the ways of Asa his father; he turned not aside from it, doing \emph{that} \emph{which} \emph{was} right in the eyes of the LORD: nevertheless the high places were not taken away; \emph{for} the people offered and burnt incense yet in the high places."],
['1Kings','1Ki','22','044','44',"And Jehoshaphat made peace with the king of Israel."],
['1Kings','1Ki','22','045','45',"Now the rest of the acts of Jehoshaphat, and his might that he shewed, and how he warred, \emph{are} they not written in the book of the chronicles of the kings of Judah?"],
['1Kings','1Ki','22','046','46',"And the remnant of the sodomites, which remained in the days of his father Asa, he took out of the land."],
['1Kings','1Ki','22','047','47',"\emph{There} \emph{was} then no king in Edom: a deputy \emph{was} king."],
['1Kings','1Ki','22','048','48',"Jehoshaphat made ships of Tharshish to go to Ophir for gold: but they went not; for the ships were broken at Ezion-geber."],
['1Kings','1Ki','22','049','49',"Then said Ahaziah the son of Ahab unto Jehoshaphat, Let my servants go with thy servants in the ships. But Jehoshaphat would not."],
['1Kings','1Ki','22','050','50',"And Jehoshaphat slept with his fathers, and was buried with his fathers in the city of David his father: and Jehoram his son reigned in his stead."],
['1Kings','1Ki','22','051','51',"Ahaziah the son of Ahab began to reign over Israel in Samaria the seventeenth year of Jehoshaphat king of Judah, and reigned two years over Israel."],
['1Kings','1Ki','22','052','52',"And he did evil in the sight of the LORD, and walked in the way of his father, and in the way of his mother, and in the way of Jeroboam the son of Nebat, who made Israel to sin:"],
['1Kings','1Ki','22','053','53',"For he served Baal, and worshipped him, and provoked to anger the LORD God of Israel, according to all that his father had done."]
]
def main():
    ProcessVerses('FirstKings22',FirstKings22)
if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{And they continued three years without war between Syria and Israel.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And it came to pass in the third year, that Jehoshaphat the king of Judah came down to the king of Israel.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{And the king of Israel said unto his servants, Know ye that Ramoth in Gilead \emph{is} our's, and we \emph{be} still, \emph{and} take it not out of the hand of the king of Syria?}
[4] \textcolor[cmyk]{0.99998,1,0,0}{And he said unto Jehoshaphat, Wilt thou go with me to battle to Ramoth-gilead? And Jehoshaphat said to the king of Israel, I \emph{am} as thou \emph{art}, my people as thy people, my horses as thy horses.}
[5] \textcolor[cmyk]{0.99998,1,0,0}{And Jehoshaphat said unto the king of Israel, Enquire, I pray thee, at the wor

great & 1 \\ \hline
save & 1 \\ \hline
only & 1 \\ \hline
Surely & 1 \\ \hline
fight & 1 \\ \hline
cried & 1 \\ \hline
perceived & 1 \\ \hline
pursuing & 1 \\ \hline
\emph{certain} & 1 \\ \hline
drew & 1 \\ \hline
bow & 1 \\ \hline
venture & 1 \\ \hline
joints & 1 \\ \hline
harness & 1 \\ \hline
wherefore & 1 \\ \hline
driver & 1 \\ \hline
Turn & 1 \\ \hline
thine & 1 \\ \hline
am & 1 \\ \hline
wounded & 1 \\ \hline
increased & 1 \\ \hline
stayed & 1 \\ \hline
even & 1 \\ \hline
ran & 1 \\ \hline
wound & 1 \\ \hline
midst & 1 \\ \hline
proclamation & 1 \\ \hline
throughout & 1 \\ \hline
going & 1 \\ \hline
sun & 1 \\ \hline
Every & 1 \\ \hline
own & 1 \\ \hline
country & 1 \\ \hline
brought & 1 \\ \hline
\emph{one} & 1 \\ \hline
pool & 1 \\ \hline
dogs & 1 \\ \hline
licked & 1 \\ \hline
armour & 1 \\ \hline
ivory & 1 \\ \hline
cities & 1 \\ \hline
built & 1 \\ \hline
fourth & 1 \\ \hline
old & 1 \\ \hline
twenty & 1 \\ \hline
Jerusalem & 1 \\ \hline
mother's & 1 \\ \hline
Azubah & 1 \\

wound & 1 \\ \hline
wounded & 1 \\ \hline
written & 2 \\ \hline
ye & 1 \\ \hline
year & 3 \\ \hline
years & 4 \\ \hline
yet & 2 \\ \hline
you & 1 \\ \hline
\end{longtable}
\end{center}



\normalsize



\subsection{Word Lengths in Chapter}
\normalsize
\begin{longtable}{l|p{3.75in}}
\caption[Words by Length in FirstKings 22]{Words by Length in FirstKings 22} \label{table:WordsIn-FirstKings-22} \\ 
\hline \multicolumn{1}{|c|}{\textbf{Length}} & \multicolumn{1}{c|}{\textbf{Words}} \\ \hline 
\endfirsthead
 
\multicolumn{2}{c}
{{\bfseries \tablename\ \thetable{} -- continued from previous page}} \\ 
\hline \multicolumn{1}{|c|}{\textbf{Length}} & \multicolumn{1}{c|}{\textbf{Words}} \\ \hline 
\endhead
 
\hline \multicolumn{2}{|r|}{{Continued if needed}} \\ \hline
\endfoot
 
\hline \hline
\endlastfoot
1 & I, a, O \\ \hline
2 & it, to, in, of, ye, \emph{is}, we, \emph{be}, he, go, me, \emph{am}, as, my, at, or, Go, up, \emph{it}, \emph{Is}, by, so, an, on, be, \emph{As}, So, no, do, If, am \\